In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scipy.stats import linregress
from matplotlib.ticker import AutoMinorLocator
# import statsmodels.api as sm
from scipy.optimize import curve_fit
import subprocess
import os
# to upgrade pandas:
# /opt/homebrew/bin/python3.10 -m pip install --upgrade pandas   


In [4]:
df = pd.read_csv('40Ca/40Ca.dat', sep='\s+', header=None, names=['Z', 'A','E0','ThetaDeg','nu','cross','error','citation'])

In [5]:
df

,Z,A,E0,ThetaDeg,nu,cross,error,citation
0,20,40,0.120,90.0,0.0075,216300.0,14430.00,Meziani:1984is
1,20,40,0.120,90.0,0.0125,92090.0,8187.00,Meziani:1984is
2,20,40,0.120,90.0,0.0175,138400.0,10410.00,Meziani:1984is
3,20,40,0.120,90.0,0.0225,123600.0,10380.00,Meziani:1984is
4,20,40,0.120,90.0,0.0275,92290.0,9483.00,Meziani:1984is
...,...,...,...,...,...,...,...,...
1334,20,40,0.841,45.5,0.3994,10280.0,67.92,Williamson:1997
1335,20,40,0.841,45.5,0.4471,10840.0,74.07,Williamson:1997
1336,20,40,0.841,45.5,0.4921,10160.0,77.69,Williamson:1997
1337,20,40,0.841,45.5,0.5355,9122.0,79.60,Williamson:1997


In [10]:
# April 12 presets
# March 1 new presets

mass_nucleon = 0.938273
mass_C12=11.178
alpha_fine = 1/137
initial_correction = 1.0

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

# q	RT photo
# 0.991	0.0371
# 1.619	0.0414
# 1.921	0.0479
# 2.213	0.0542
# 2.500	0.0603
# 2.783	0.0664
# 3.500	0.0817
qvcenter_to_RTphoto = {0.991:0.0371,1.619:0.0414,1.921:0.0479,2.213:0.0542,2.500:0.0603,2.783:0.0664,3.500:0.0817}

# Q2centers = [0.01 , 0.02 , 0.026, 0.04 , 0.056, 0.093, 0.12 , 0.16 , 0.265, 0.38 , 0.5  , 0.65 , 0.8  , 1.05 , 1.25 , 1.75 , 2.25 , 2.75 , 3.25 , 3.75 ]
# Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.209, 0.322, 0.438, 0.562, 0.738, 0.962, 1.150, 1.500, 2.000, 2.500, 
#           3.000, 3.500, 4.000]
# Q2bin_names = ['0.004~0.015', '0.015~0.025', '0.025~0.035', '0.035~0.045', '0.045~0.070', '0.070~0.100', '0.100~0.145', '0.145~0.209','0.209~0.322', 
#         '0.322~0.438', '0.438~0.562', '0.562~0.738','0.738~0.962', '0.962~1.150', '1.150~1.500', '1.500~2.000','2.000~2.500', '2.500~3.000', '3.000~3.500', '3.500~4.000'],

# qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.302, 1.619, 1.921, 2.213, 2.500, 2.783]
# qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.168, 1.460, 1.770, 2.067, 2.357, 2.642, 2.923]
# qvbin_names = ['0.063~0.124','0.124~0.158','0.158~0.186','0.186~0.223','0.223~0.270','0.270~0.340','0.340~0.428','0.428~0.523','0.523~0.609',
#                '0.609~0.702','0.702~0.878','0.878~1.168','1.168~1.460','1.460~1.770','1.770~2.067','2.067~2.357','2.357~2.642','2.642~2.923']

qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# qvbin_names = ['0.063~0.124','0.124~0.158','0.158~0.186','0.186~0.223','0.223~0.270','0.270~0.340','0.340~0.428','0.428~0.523','0.523~0.609',
#                 '0.609~0.702','0.702~0.878','0.878~1.302','1.302~1.770','1.770~2.067','2.067~2.357','2.357~2.642','2.642~2.923','2.923~4.500']
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# Q2bin_names = ['0.004~0.015', '0.015~0.025', '0.025~0.035', '0.035~0.045', '0.045~0.070', '0.070~0.100', '0.100~0.145', '0.145~0.206', '0.206~0.322', 
#                '0.322~0.438', '0.438~0.650', '0.650~1.050', '1.050~1.500', '1.500~2.000', '2.000~2.500', '2.500~3.000', '3.000~3.500', '3.500~4.000']
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']



# nuwidths = [0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01]
nuwidths = [0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.005, 0.005, 0.005, 
            0.01,     0.01,   0.01,   0.01,   0.01,  0.01,  0.01, 0.01, 
            0.04, 0.04, 0.04, 0.04]

W2widths = [0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04]

dataSet_to_name = {1:"Barreau:1983ht", 2:"O'Connell:198", 3:"Sealock:1989nx", 4:"Baran:1988tw", 5:"Bagdasaryan:1988hp", 6:"Dai - HallA:2019da", 
    7:"Arrington:1995hs", 8:"Day:1993md", 9:"Arrington:1998psnoCC", 10:"Gaskell:2008", 11:"Whitney:1974hr", 12:"AlsamiJan05", 13:"VaheJun07", 
    14:"Gomez74", 15:"Fomin", 16:"Yamaguchi73", 17:"Ryan84", 18:"Cyzyk:1963zz", 19:"Bounin63", 20:"Photo-Daphne", 21:"Spamer70", 22:"Goldemberg64", 
    23:"DeForrest65", 24:"Donnelly68", 25:"Barreau:French", 26:"Julia:Genie"}

dataSet_to_normError = {1:0.24320E-02, 2:0.86008E-02, 3:0.48305E-02, 4:0.45501E-02, 5:0.82797E-02, 6:0.52969E-02, 7:0.13229E-01, 8:0.33307E-02, 
        9:0.34432E-02, 10:0.50742E-02, 11:0.15258E-01, 12:0.67079E-03, 13:0.69905E-03, 14:0.14875E-01, 15:0.30856E-02, 16:0.29024E-02, 
        17:0.13049E-01, 18:0.1, 19:0.23, 20:0.10513, 21:0.4, 22:0.1, 23:0.1, 24:0.1, 25:0.002432, 26:0}

dataSet_to_normalization = {1:0.99185, 2:0.97869, 3:1.0315, 4:0.99241, 5:0.98777, 6:1.0108, 7:0.97427, 8:1.0071, 9:0.98884, 10:0.99340, 11:1.0149, 
                12:0.99812, 13:1.0029, 14:1.0125, 15:1.0046, 16:1.0019, 17:1.0517, 18:1.0, 19:1.1500, 20:0.99754, 21:1.2, 22:1.1, 23:0.85, 24:1.0, 
                25:0.99185, 26:1.0}

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)
def linear_model(x, a, b):
    return a * x + b


# # March 1 new presets

# mass_nucleon = 0.938273
# mass_C12=11.178
# alpha_fine = 1/137
# initial_correction = 1.0

# def round_sig_3(x):
#     return '%s' % float('%.3g' % x)

# # q	RT photo
# # 0.991	0.0371
# # 1.619	0.0414
# # 1.921	0.0479
# # 2.213	0.0542
# # 2.500	0.0603
# # 2.783	0.0664
# # 3.500	0.0817
# qvcenter_to_RTphoto = {0.991:0.0371,1.619:0.0414,1.921:0.0479,2.213:0.0542,2.500:0.0603,2.783:0.0664,3.500:0.0817}

# # Q2centers = [0.01 , 0.02 , 0.026, 0.04 , 0.056, 0.093, 0.12 , 0.16 , 0.265, 0.38 , 0.5  , 0.65 , 0.8  , 1.05 , 1.25 , 1.75 , 2.25 , 2.75 , 3.25 , 3.75 ]
# # Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.209, 0.322, 0.438, 0.562, 0.738, 0.962, 1.150, 1.500, 2.000, 2.500, 
# #           3.000, 3.500, 4.000]
# # Q2bin_names = ['0.004~0.015', '0.015~0.025', '0.025~0.035', '0.035~0.045', '0.045~0.070', '0.070~0.100', '0.100~0.145', '0.145~0.209','0.209~0.322', 
# #         '0.322~0.438', '0.438~0.562', '0.562~0.738','0.738~0.962', '0.962~1.150', '1.150~1.500', '1.500~2.000','2.000~2.500', '2.500~3.000', '3.000~3.500', '3.500~4.000'],

# # qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.302, 1.619, 1.921, 2.213, 2.500, 2.783]
# # qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.168, 1.460, 1.770, 2.067, 2.357, 2.642, 2.923]
# # qvbin_names = ['0.063~0.124','0.124~0.158','0.158~0.186','0.186~0.223','0.223~0.270','0.270~0.340','0.340~0.428','0.428~0.523','0.523~0.609',
# #                '0.609~0.702','0.702~0.878','0.878~1.168','1.168~1.460','1.460~1.770','1.770~2.067','2.067~2.357','2.357~2.642','2.642~2.923']

# qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# # qvbin_names = ['0.063~0.124','0.124~0.158','0.158~0.186','0.186~0.223','0.223~0.270','0.270~0.340','0.340~0.428','0.428~0.523','0.523~0.609',
# #                 '0.609~0.702','0.702~0.878','0.878~1.302','1.302~1.770','1.770~2.067','2.067~2.357','2.357~2.642','2.642~2.923','2.923~4.500']
# qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
#                 '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# # Q2bin_names = ['0.004~0.015', '0.015~0.025', '0.025~0.035', '0.035~0.045', '0.045~0.070', '0.070~0.100', '0.100~0.145', '0.145~0.206', '0.206~0.322', 
# #                '0.322~0.438', '0.438~0.650', '0.650~1.050', '1.050~1.500', '1.500~2.000', '2.000~2.500', '2.500~3.000', '3.000~3.500', '3.500~4.000']
# Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
#                 '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']



# # nuwidths = [0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.01, 0.01, 0.01, 0.01, 0.01,0.01, 0.01, 0.01, 0.01, 0.01]
# nuwidths = [0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.005, 0.005, 0.005, 
#             0.01,     0.01,   0.01,   0.01,   0.01,  0.01,  0.01, 0.01, 
#             0.04, 0.04, 0.04, 0.04]

# W2widths = [0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04]

# dataSet_to_name = {1:"Barreau:1983ht", 2:"O'Connell:198", 3:"Sealock:1989nx", 4:"Baran:1988tw", 5:"Bagdasaryan:1988hp", 6:"Dai - HallA:2019da", 
#     7:"Arrington:1995hs", 8:"Day:1993md", 9:"Arrington:1998psnoCC", 10:"Gaskell:2008", 11:"Whitney:1974hr", 12:"AlsamiJan05", 13:"VaheJun07", 
#     14:"Gomez74", 15:"Fomin", 16:"Yamaguchi73", 17:"Ryan84", 18:"Cyzyk:1963zz", 19:"Bounin63", 20:"Photo-Daphne", 21:"Spamer70", 22:"Goldemberg64", 
#     23:"DeForrest65", 24:"Donnelly68",25:"Barreau"}

# dataSet_to_normError = {1:0.24320E-02, 2:0.86008E-02, 3:0.48305E-02, 4:0.45501E-02, 5:0.82797E-02, 6:0.52969E-02, 7:0.13229E-01, 8:0.33307E-02, 
#         9:0.34432E-02, 10:0.50742E-02, 11:0.15258E-01, 12:0.67079E-03, 13:0.69905E-03, 14:0.14875E-01, 15:0.30856E-02, 16:0.29024E-02, 
#         17:0.13049E-01, 18:0.1, 19:0.23, 20:0.10513, 21:0.4, 22:0.1, 23:0.1}

# dataSet_to_normalization = {1:0.99185, 2:0.97869, 3:1.0315, 4:0.99241, 5:0.98777, 6:1.0108, 7:0.97427, 8:1.0071, 9:0.98884, 10:0.99340, 11:1.0149, 
#                 12:0.99812, 13:1.0029, 14:1.0125, 15:1.0046, 16:1.0019, 17:1.0517, 18:1.0, 19:1.1500, 20:0.99754, 21:1.2, 22:1.1, 23:0.85}

# def append_row(df, row):
#     return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)
# def linear_model(x, a, b):
#     return a * x + b


In [11]:

# df = pd.read_csv("df_fortran_Feb16.csv")
# df = pd.read_csv("df_fortran_Feb20.csv")
# df = pd.read_csv('df_fortran_Goldemberg_Feb23.csv')
# df = pd.read_csv('df_fortran_Feb24.csv')
# df = pd.read_csv('df_fortran_March1.csv')
# df = pd.read_csv('DataSet13_March15.csv')
# df = pd.read_csv('Barreau/Barreau_cross_French.csv')
# df = pd.read_csv('Barreau/Barreau_ChatGPT.csv')
# df = pd.read_csv('Sheren_April12.csv')
# df = pd.read_csv('Barreau/Barreau_cross_April12.csv')
# df = pd.read_csv('Sheren/Sheren_Data.csv')
df = pd.read_csv('Genie_All/CrossSection_C_Genie.csv')
df['dataSet'] = 26
df = df.loc[df['error']!=0.0]



In [12]:
# df.loc[df['error']==0.0]
df

,Z,A,E0,ThetaDeg,nu,cross,error,dataSet
8,6,12,0.40,71.5000,0.0340,4.42867,3.13154,26
11,6,12,0.40,71.5000,0.0460,66.43010,12.12840,26
12,6,12,0.40,71.5000,0.0500,418.51000,30.44210,26
13,6,12,0.40,71.5000,0.0540,1031.88000,47.80090,26
14,6,12,0.40,71.5000,0.0580,1755.97000,62.35630,26
...,...,...,...,...,...,...,...,...
32654,6,12,0.68,32.6667,0.5678,3169.33000,93.66210,26
32655,6,12,0.68,32.6667,0.5746,3100.13000,92.63400,26
32656,6,12,0.68,32.6667,0.5814,2441.35000,82.20450,26
32657,6,12,0.68,32.6667,0.5882,1264.96000,59.17240,26


In [13]:
# calculate H_cc, etc. from source data
# calculate thetaRad, sin2(T/2), cos2(T/2), tan2(T/2)
df["ThetaRad"]=df["ThetaDeg"]*np.pi/180
df["sin2(T/2)"]=(np.sin(df["ThetaRad"]/2))**2
df["cos2(T/2)"]=(np.cos(df["ThetaRad"]/2))**2
df["tan2(T/2)"]=(np.tan(df["ThetaRad"]/2))**2


df["normalization"]=df["dataSet"].map(dataSet_to_normalization)
df["normError"]=df["dataSet"].map(dataSet_to_normError)

# calculate normalized cross section 
# df["cross"] = df["cross"] * df["normalization"]
# df["error"] = df["error"] * df["normalization"]
df['normCross'] = df['cross'] * df['normalization']
df['normCrossError']=df['normCross']*np.sqrt(
        (df['error']/df['cross'])**2+(df['normError']/df['normalization'])**2
    )

SystemError = 0.05
df['normCrossError'] = np.sqrt(df['normCrossError']**2 + (SystemError*df['normCross'])**2)





# Ex = nu - nu_elastic
# nu = Ex + nu_elastic
# nu_elastic = E0 - E0 / (1 + 2*E0*sin2(T/2) / mass_c12)
# Ex = nu - E0 + E0 / (1 + 2*E0*sin2(T/2) / mass_c12)
df["nu_elastic"]=df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12)


# convert yamaguchi, ryan's Ex to nu:
# df.loc[df["dataSet"]==16,"nu"] = df.loc[df["dataSet"]==16,"nu"] + df.loc[df["dataSet"]==16,"nu_elastic"]
# df.loc[df["dataSet"]==17,"nu"] = df.loc[df["dataSet"]==17,"nu"] + df.loc[df["dataSet"]==17,"nu_elastic"]

df["Ex"]=df["nu"]-df["nu_elastic"]


df["Ex"] = df["nu"] - (df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12))

# R = 2.894, Z = 6, A = 12
df["R"]=1.1*(df["A"])**(1/3)+0.86/((df["A"])**(1/3))
df["Veff"]=0.775*(3/2)*alpha_fine*(df["Z"]-1)/df["R"]
# wrong formula?
df["Veff"]=0.0031


df["Eeff"]=df["E0"]+df["Veff"]

df["Eprime"]=df["E0"]-df["nu"]
df["Eprime_eff"]=df["Eprime"]+df["Veff"]

df["F2foc"]=(df["Eeff"]/df["E0"])**2

df["Q2"]=4*df["E0"]*(df["Eprime"])*df["sin2(T/2)"]
df["Q2eff"]=4*df["Eeff"]*df["Eprime_eff"]*df["sin2(T/2)"]
# df["qeff"]=df["Q2eff"]+df["nu"]**2


# df["q3momt_squared"]=df["nu"]**2+df["Q2"]
# qeff2
df["q3momt_squared"]=df["nu"]**2+df["Q2eff"]
# qeff
df["q3momt"]=np.sqrt(df["q3momt_squared"])

# df["W2"]=mass_nucleon**2+2*mass_nucleon*df["nu"]-df["Q2"]
# W2eff
df["W2"]=mass_nucleon**2+2*mass_nucleon*df["nu"]-df["Q2eff"]

# df["W"]=np.sqrt(df["W2"])

# df["epsilon"]=1/(1+2*(1+(df["nu"]**2)/df["Q2"])*df["tan2(T/2)"])
# epsilon effective
df["epsilon"]=1/(1+2*(1+(df["nu"]**2)/df["Q2eff"])*df["tan2(T/2)"]) 


## QUESTIONS!!!!!
# df["gamma"]=alpha_fine*df["Eprime"]*(df["W2"]-mass_nucleon**2)/((4*((np.pi)**2)*df["Q2"]*mass_nucleon*df["E0"])*(1-df["epsilon"]))
df["gamma"]=alpha_fine*df["Eprime_eff"]*(df["W2"]-mass_nucleon**2)/((4*((np.pi)**2)*df["Q2eff"]*mass_nucleon*df["E0"])*(1-df["epsilon"]))

# df["Sig_R"]=df["cross"]/df["gamma"]
df["Sig_R"]=df["normCross"]/df["gamma"]
df["D_sig_R"]=df["error"]/df["gamma"]

df["Sig_mott"]=4*(alpha_fine**2)*(df["Eprime"]**2)*df["cos2(T/2)"]/(df["Q2"]**2)
# Sig_mott_eff = Sig_mott / F2eff
df["Sig_mott_eff"]=df["Sig_mott"]*df["E0"]/df["Eeff"]

# df["H"]=(df["q3momt_squared"]**2)/(4*(alpha_fine**2)*(df["Eprime"]**2)*(df["cos2(T/2)"]+2*(df["q3momt_squared"]/df["Q2"])*df["sin2(T/2)"]))
df["H"]=(df["q3momt_squared"]**2)/(4*(alpha_fine**2)*(df["Eprime_eff"]**2)*(df["cos2(T/2)"]+2*(df["q3momt_squared"]/df["Q2eff"])*df["sin2(T/2)"]))

# H with coloumb correction
df["Hcc"]=df["H"] / df["F2foc"]


# H without coloumb correction
# df["Hstar_Sig(nb)"]=initial_correction*df["H"]*df["cross"]
# df["Hstar_error(nb)"]=initial_correction*df["H"]*df["error"]
df["Hstar_Sig(nb)"]=initial_correction*df["H"]*df["normCross"]
df["Hstar_error(nb)"]=initial_correction*df["H"]*df["normCrossError"]
df["Hstar_Sig(GeV)"]=df["Hstar_Sig(nb)"]/((0.1973269**2)*10000000)
df["Hstar_error(GeV)"]=df["Hstar_error(nb)"]/((0.1973269**2)*10000000)

# H with coloumb correction
# df["Hcc_Sig(nb)"]=initial_correction*df["Hcc"]*df["cross"]
# df["Hcc_error(nb)"]=initial_correction*df["Hcc"]*df["error"]
df["Hcc_Sig(nb)"]=initial_correction*df["Hcc"]*df["normCross"]
df["Hcc_error(nb)"]=initial_correction*df["Hcc"]*df["normCrossError"]
df["Hcc_Sig(GeV)"]=df["Hcc_Sig(nb)"]/((0.1973269**2)*10000000)
df["Hcc_error(GeV)"]=df["Hcc_error(nb)"]/((0.1973269**2)*10000000)

df['qvbin'] = 0
df['qvcenter'] = 0
df['Q2bin'] = 0
df['Q2center'] = 0

df["qvbin"]=pd.cut(x=df["q3momt"],bins=qvbins,labels=qvbin_names,right=True)
df["qvcenter"]=pd.cut(x=df["q3momt"],bins=qvbins,labels=qvcenters,right=True)
df['qvcenter']=pd.to_numeric(df['qvcenter'])

df["Q2bin"]=pd.cut(x=df["Q2"],bins=Q2bins,labels=Q2bin_names,right=True)
df["Q2center"]=pd.cut(x=df["Q2"],bins=Q2bins,labels=Q2centers,right=True)
df['Q2center']=pd.to_numeric(df['Q2center'])


In [14]:
# compute Fortran fit RL, RT for the dataset (qv center, qv data) analysis in nu
df['RL_fortran_qvcenter_nu']=0
df['RT_fortran_qvcenter_nu']=0
df['RL_fortran_qvdata_nu']=0
df['RT_fortran_qvdata_nu']=0

center_errors = []
data_errors = []
fortran_folder = "Eric/response/"
# for index, row in df.iterrows():
for qvcenter in qvcenters:
    picked = df.loc[df['qvcenter']==qvcenter]
    for index, row in picked.iterrows():
        # calculate center
#         qvcenter = row['qvcenter']
        nu = row['nu']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qvcenter)+'\n'+str(nu)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. qvcenter, nu:',qvcenter,nu)
            center_errors.append(index)
            continue
    #     qv,q2,ex,nu,RTTOT,RLTOT,RTQE,RLQE,RTIE,RLIE,RTE,RLE,RTNS,RLNS
        ex_center = center_output[2]
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_qvcenter_nu']=RL_center
        df.loc[index,'RT_fortran_qvcenter_nu']=RT_center



        # calculate data
        qv = row['q3momt']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qv)+'\n'+str(nu)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. qv, nu:',qv,nu)
            data_errors.append(index)
            continue
        ex_data = data_output[2]
        RT_data = data_output[4]
        RL_data = data_output[5]

        print('index:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)
    #     print('index:',index,'Ex:',Ex,'ex_data:',ex_data,'Ex-ex_data:',Ex-ex_data )
        df.loc[index,'RL_fortran_qvdata_nu']=RL_data
        df.loc[index,'RT_fortran_qvdata_nu']=RT_data




index: 2295 RLRT_center: 0.0309 0.0112 RLRT_data: 0.0332 0.0122
index: 2296 RLRT_center: 0.0295 0.0108 RLRT_data: 0.0319 0.0119
index: 2297 RLRT_center: 0.0277 1670.0 RLRT_data: 0.0301 0.0115
index: 2298 RLRT_center: 0.0259 0.00961 RLRT_data: 0.0282 0.0109


/var/folders/74/13k54tvs74vchflrfsmkcc_m0000gn/T/ipykernel_20208/2846308659.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0309' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index,'RL_fortran_qvcenter_nu']=RL_center
/var/folders/74/13k54tvs74vchflrfsmkcc_m0000gn/T/ipykernel_20208/2846308659.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0112' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index,'RT_fortran_qvcenter_nu']=RT_center
/var/folders/74/13k54tvs74vchflrfsmkcc_m0000gn/T/ipykernel_20208/2846308659.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0332' has dtype incompatible with int64, please explicitly cast to a c

ValueError: could not convert string to float: '0.337+149'

In [10]:
# compute Fortran fit RL, RT for the dataset (qv center, qv data), analysis in Ex
df['RL_fortran_qvcenter_ex']=0
df['RT_fortran_qvcenter_ex']=0
df['RL_fortran_qvdata_ex']=0
df['RT_fortran_qvdata_ex']=0

center_errors = []
data_errors = []
fortran_folder = "Eric/response/"
# for index, row in df.iterrows():
for qvcenter in qvcenters:
    picked = df.loc[df['qvcenter']==qvcenter]
    for index, row in picked.iterrows():
        # calculate center
#         qvcenter = row['qvcenter']
        Ex = row['Ex']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qvcenter)+'\n'+str(Ex)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. qvcenter, Ex:',qvcenter,Ex)
            center_errors.append(index)
            continue
    #     qv,q2,ex,nu,RTTOT,RLTOT,RTQE,RLQE,RTIE,RLIE,RTE,RLE,RTNS,RLNS
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_qvcenter_ex']=RL_center
        df.loc[index,'RT_fortran_qvcenter_ex']=RT_center


        # calculate data
        qv = row['q3momt']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qv)+'\n'+str(Ex)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. qv, Ex:',qv,Ex)
            data_errors.append(index)
            continue
        RT_data = data_output[4]
        RL_data = data_output[5]

        print('index:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data,'Ex:',Ex)
        df.loc[index,'RL_fortran_qvdata_ex']=RL_data
        df.loc[index,'RT_fortran_qvdata_ex']=RT_data



index: 0 RLRT_center: 0.000342 0.00227 RLRT_data: 3.12e-05 0.000388 Ex: 0.13335378725647126
index: 1 RLRT_center: 0.000545 0.00326 RLRT_data: 5.58e-05 0.000592 Ex: 0.14885378725647128
index: 2 RLRT_center: 0.000815 0.00449 RLRT_data: 9.81e-05 0.000891 Ex: 0.16425378725647127
index: 3 RLRT_center: 0.00114 0.00587 RLRT_data: 0.000169 0.00133 Ex: 0.17975378725647126
index: 4 RLRT_center: 0.00149 0.00731 RLRT_data: 0.000278 0.00193 Ex: 0.19525378725647127
index: 5 RLRT_center: 0.00184 0.00869 RLRT_data: 0.000431 0.00271 Ex: 0.21075378725647126
index: 6 RLRT_center: 0.00216 0.00992 RLRT_data: 0.000626 0.00364 Ex: 0.22625378725647127
index: 7 RLRT_center: 0.00245 0.0109 RLRT_data: 0.000847 0.00466 Ex: 0.2417537872564713
index: 8 RLRT_center: 0.00267 0.0116 RLRT_data: 0.00108 0.00568 Ex: 0.25725378725647124
index: 9 RLRT_center: 0.00284 0.0117 RLRT_data: 0.0017 0.00822 Ex: 0.3037537872564713
index: 10 RLRT_center: 0.00276 0.0112 RLRT_data: 0.00184 0.00873 Ex: 0.3192537872564713
index: 11 RLRT

index: 72 RLRT_center: 0.00125 0.00659 RLRT_data: 0.000443 0.00285 Ex: 0.4802661278556939
index: 73 RLRT_center: 0.00127 0.00667 RLRT_data: 0.000497 0.00314 Ex: 0.49186612785569384
index: 74 RLRT_center: 0.0013 0.00691 RLRT_data: 0.000599 0.00368 Ex: 0.5149661278556938
index: 75 RLRT_center: 0.00134 0.0071 RLRT_data: 0.000644 0.00392 Ex: 0.5265661278556938
index: 76 RLRT_center: 0.00138 0.00734 RLRT_data: 0.000682 0.00413 Ex: 0.5380661278556939
index: 77 RLRT_center: 0.00143 0.00766 RLRT_data: 0.000717 0.00432 Ex: 0.5496661278556938
index: 78 RLRT_center: 0.00151 0.00802 RLRT_data: 0.000748 0.00449 Ex: 0.5611661278556939
index: 79 RLRT_center: 0.0016 0.00845 RLRT_data: 0.000777 0.00465 Ex: 0.5727661278556938
index: 80 RLRT_center: 0.0017 0.00892 RLRT_data: 0.000806 0.00481 Ex: 0.5842661278556939
index: 81 RLRT_center: 0.00195 0.00995 RLRT_data: 0.000869 0.00515 Ex: 0.6073661278556939
index: 82 RLRT_center: 0.0021 0.0105 RLRT_data: 0.00091 0.00534 Ex: 0.6189661278556938
index: 83 RLRT_c

index: 444 RLRT_center: 0.00262 0.0121 RLRT_data: 0.00215 0.0103 Ex: 0.6554557891840518
index: 445 RLRT_center: 0.00295 0.0128 RLRT_data: 0.00228 0.0105 Ex: 0.6746557891840518
index: 446 RLRT_center: 0.00331 0.0133 RLRT_data: 0.00242 0.0107 Ex: 0.6938557891840518
index: 447 RLRT_center: 0.00373 0.0138 RLRT_data: 0.00257 0.0109 Ex: 0.7130557891840518
index: 448 RLRT_center: 0.00425 0.0142 RLRT_data: 0.00272 0.011 Ex: 0.7322557891840518
index: 449 RLRT_center: 0.00489 0.0146 RLRT_data: 0.0029 0.011 Ex: 0.7514557891840518
index: 450 RLRT_center: 0.00571 0.0151 RLRT_data: 0.00309 0.0111 Ex: 0.7706557891840518
index: 451 RLRT_center: 0.00675 0.0158 RLRT_data: 0.0033 0.0111 Ex: 0.7898557891840517
index: 452 RLRT_center: 0.00804 0.0168 RLRT_data: 0.00354 0.0111 Ex: 0.8090557891840517
index: 453 RLRT_center: 0.00958 0.0181 RLRT_data: 0.0038 0.0112 Ex: 0.8282557891840517
index: 454 RLRT_center: 0.0114 0.0197 RLRT_data: 0.00409 0.0112 Ex: 0.8474557891840517
index: 455 RLRT_center: 0.0133 0.0217 

index: 924 RLRT_center: 0.000788 0.00444 RLRT_data: 2.58e-06 6.63e-05 Ex: 0.36062570221335566
index: 110 RLRT_center: 0.00162 0.00483 RLRT_data: 0.0159 0.0193 Ex: 1.1439626635873632
index: 111 RLRT_center: 0.00193 0.00526 RLRT_data: 0.0166 0.0197 Ex: 1.1624626635873632
index: 112 RLRT_center: 0.00228 0.00573 RLRT_data: 0.0173 0.0201 Ex: 1.1809626635873631
index: 113 RLRT_center: 0.0027 0.00625 RLRT_data: 0.0181 0.0205 Ex: 1.1995626635873633
index: 114 RLRT_center: 0.00318 0.0068 RLRT_data: 0.0189 0.0208 Ex: 1.2180626635873633
index: 115 RLRT_center: 0.00372 0.00741 RLRT_data: 0.0197 0.0212 Ex: 1.2365626635873632
index: 116 RLRT_center: 0.00435 0.00807 RLRT_data: 0.0205 0.0215 Ex: 1.2550626635873632
index: 117 RLRT_center: 0.00506 0.00879 RLRT_data: 0.0214 0.0219 Ex: 1.2735626635873631
index: 118 RLRT_center: 0.00587 0.00959 RLRT_data: 0.0223 0.0222 Ex: 1.292162663587363
index: 119 RLRT_center: 0.00679 0.0104 RLRT_data: 0.0233 0.0225 Ex: 1.3106626635873633
index: 120 RLRT_center: 0.0078

index: 250 RLRT_center: 0.000202 0.00149 RLRT_data: 0.000101 0.00083 Ex: 0.9022720997841984
index: 251 RLRT_center: 0.000221 0.00159 RLRT_data: 0.00011 0.000893 Ex: 0.9145720997841984
index: 252 RLRT_center: 0.000241 0.0017 RLRT_data: 0.00012 0.000957 Ex: 0.9268720997841984
index: 253 RLRT_center: 0.000263 0.00181 RLRT_data: 0.00013 0.00102 Ex: 0.9391720997841984
index: 254 RLRT_center: 0.000287 0.00193 RLRT_data: 0.00014 0.00109 Ex: 0.9514720997841983
index: 255 RLRT_center: 0.000315 0.00204 RLRT_data: 0.000151 0.00115 Ex: 0.9637720997841983
index: 256 RLRT_center: 0.000347 0.00217 RLRT_data: 0.000163 0.00121 Ex: 0.9760720997841985
index: 257 RLRT_center: 0.000383 0.0023 RLRT_data: 0.000175 0.00128 Ex: 0.9883720997841985
index: 258 RLRT_center: 0.000425 0.00244 RLRT_data: 0.000188 0.00135 Ex: 1.0006720997841985
index: 259 RLRT_center: 0.000473 0.00259 RLRT_data: 0.000201 0.00141 Ex: 1.0129720997841984
index: 260 RLRT_center: 0.000528 0.00275 RLRT_data: 0.000216 0.00148 Ex: 1.025272099

...

index: 1101 RLRT_center: 0.00055 0.00218 RLRT_data: 0.000415 0.00181 Ex: 1.305729998636796
index: 1102 RLRT_center: 0.000651 0.00239 RLRT_data: 0.000449 0.00188 Ex: 1.322129998636796
index: 1103 RLRT_center: 0.000768 0.00263 RLRT_data: 0.000485 0.00196 Ex: 1.338529998636796
index: 1104 RLRT_center: 0.000906 0.00289 RLRT_data: 0.000525 0.00203 Ex: 1.354929998636796
index: 1105 RLRT_center: 0.00107 0.00317 RLRT_data: 0.000567 0.00211 Ex: 1.371429998636796
index: 1106 RLRT_center: 0.00125 0.00347 RLRT_data: 0.000611 0.00219 Ex: 1.387829998636796
index: 1107 RLRT_center: 0.00146 0.0038 RLRT_data: 0.000659 0.00227 Ex: 1.4042299986367959
index: 1108 RLRT_center: 0.00171 0.00416 RLRT_data: 0.000709 0.00236 Ex: 1.420729998636796
index: 1109 RLRT_center: 0.00198 0.00454 RLRT_data: 0.000762 0.00245 Ex: 1.437129998636796
index: 1110 RLRT_center: 0.0023 0.00496 RLRT_data: 0.000818 0.00254 Ex: 1.453529998636796
index: 1111 RLRT_center: 0.00265 0.0054 RLRT_data: 0.000877 0.00263 Ex: 1.46992999863679

index: 685 RLRT_center: 0.0131 0.0128 RLRT_data: 0.00567 0.00767 Ex: 1.9410478591546396
index: 686 RLRT_center: 0.0146 0.0136 RLRT_data: 0.0059 0.00783 Ex: 1.9558478591546398
index: 699 RLRT_center: 0.000116 0.000676 RLRT_data: 7.11e-05 0.000464 Ex: 1.4236593277265985
index: 700 RLRT_center: 0.000132 0.000736 RLRT_data: 7.71e-05 0.00049 Ex: 1.4360593277265987
index: 701 RLRT_center: 0.000151 0.000802 RLRT_data: 8.35e-05 0.000516 Ex: 1.4485593277265987
index: 702 RLRT_center: 0.000172 0.000873 RLRT_data: 9.04e-05 0.000542 Ex: 1.4610593277265986
index: 703 RLRT_center: 0.000197 0.000949 RLRT_data: 9.78e-05 0.00057 Ex: 1.4734593277265986
index: 704 RLRT_center: 0.000225 0.00103 RLRT_data: 0.000106 0.000599 Ex: 1.4859593277265986
index: 705 RLRT_center: 0.000257 0.00112 RLRT_data: 0.000114 0.000628 Ex: 1.4984593277265987
index: 706 RLRT_center: 0.000293 0.00122 RLRT_data: 0.000124 0.000659 Ex: 1.5108593277265987
index: 707 RLRT_center: 0.000335 0.00133 RLRT_data: 0.000134 0.000691 Ex: 1.52

index: 1199 RLRT_center: 2.14e-05 0.00019 RLRT_data: 2.42e-05 0.000211 Ex: 1.2718344919707993
index: 1200 RLRT_center: 2.58e-05 0.000221 RLRT_data: 2.68e-05 0.000229 Ex: 1.2864344919707993
index: 1201 RLRT_center: 3.08e-05 0.000255 RLRT_data: 2.96e-05 0.000247 Ex: 1.3010344919707992
index: 1202 RLRT_center: 3.65e-05 0.000292 RLRT_data: 3.25e-05 0.000265 Ex: 1.3156344919707992
index: 1203 RLRT_center: 4.3e-05 0.000332 RLRT_data: 3.57e-05 0.000284 Ex: 1.3301344919707991
index: 1204 RLRT_center: 5.05e-05 0.000375 RLRT_data: 3.9e-05 0.000303 Ex: 1.3447344919707993
index: 1205 RLRT_center: 5.92e-05 0.000421 RLRT_data: 4.27e-05 0.000323 Ex: 1.3593344919707993
index: 1206 RLRT_center: 6.91e-05 0.000472 RLRT_data: 4.65e-05 0.000343 Ex: 1.3739344919707992
index: 1207 RLRT_center: 8.06e-05 0.000526 RLRT_data: 5.07e-05 0.000364 Ex: 1.3885344919707991
index: 1208 RLRT_center: 9.38e-05 0.000585 RLRT_data: 5.51e-05 0.000385 Ex: 1.4030344919707993
index: 1209 RLRT_center: 9.38e-05 0.000585 RLRT_data:

index: 774 RLRT_center: 0.00169 0.0034 RLRT_data: 0.000542 0.0015 Ex: 1.95250845252083
index: 775 RLRT_center: 0.00187 0.00363 RLRT_data: 0.000575 0.00156 Ex: 1.9639084525208297
index: 1045 RLRT_center: 0.0055 0.00725 RLRT_data: 0.0145 0.0131 Ex: 2.0965257022133557
index: 1046 RLRT_center: 0.00631 0.00788 RLRT_data: 0.0149 0.0133 Ex: 2.1144257022133557
index: 1047 RLRT_center: 0.00723 0.00855 RLRT_data: 0.0153 0.0134 Ex: 2.1323257022133557
index: 1048 RLRT_center: 0.00827 0.00927 RLRT_data: 0.0157 0.0136 Ex: 2.1502257022133557
index: 1049 RLRT_center: 0.00945 0.01 RLRT_data: 0.0161 0.0138 Ex: 2.1681257022133558
index: 1050 RLRT_center: 0.0108 0.0108 RLRT_data: 0.0166 0.014 Ex: 2.186025702213356
index: 1051 RLRT_center: 0.0123 0.0117 RLRT_data: 0.017 0.0141 Ex: 2.203925702213356
index: 1144 RLRT_center: 0.00119 0.00269 RLRT_data: 0.00369 0.00585 Ex: 1.913529998636796
index: 1145 RLRT_center: 0.00138 0.00297 RLRT_data: 0.00385 0.00599 Ex: 1.929929998636796
index: 1146 RLRT_center: 0.0016

index: 802 RLRT_center: 0.00352 0.00518 RLRT_data: 0.00244 0.00403 Ex: 2.3059084525208298
index: 803 RLRT_center: 0.00385 0.00549 RLRT_data: 0.00254 0.00413 Ex: 2.31730845252083
index: 804 RLRT_center: 0.00421 0.00581 RLRT_data: 0.00264 0.00424 Ex: 2.32870845252083
index: 805 RLRT_center: 0.00459 0.00615 RLRT_data: 0.00274 0.00434 Ex: 2.34010845252083
index: 1170 RLRT_center: 0.00314 0.00481 RLRT_data: 0.00818 0.009 Ex: 2.291429998636796
index: 1171 RLRT_center: 0.00358 0.00523 RLRT_data: 0.00841 0.00914 Ex: 2.307829998636796
index: 1172 RLRT_center: 0.00407 0.00569 RLRT_data: 0.00865 0.00928 Ex: 2.324229998636796
index: 1173 RLRT_center: 0.00461 0.00617 RLRT_data: 0.00889 0.00942 Ex: 2.3406299986367958
index: 1174 RLRT_center: 0.00522 0.00668 RLRT_data: 0.00914 0.00956 Ex: 2.357129998636796
index: 1175 RLRT_center: 0.0059 0.00722 RLRT_data: 0.00938 0.00971 Ex: 2.373529998636796
index: 1176 RLRT_center: 0.00666 0.00779 RLRT_data: 0.00963 0.00985 Ex: 2.389929998636796
index: 1177 RLRT_c

In [11]:
# compute Fortran fit RL, RT for the dataset (q2 center, q2 data) analysis in nu
df['RL_fortran_q2center_nu']=0
df['RT_fortran_q2center_nu']=0
df['RL_fortran_q2data_nu']=0
df['RT_fortran_q2data_nu']=0

center_errors = []
data_errors = []
fortran_folder = "Eric/response/"
# for index, row in df.iterrows():
for Q2center in Q2centers:
    picked = df.loc[df['Q2center']==Q2center]
    for index, row in picked.iterrows():
        # calculate center
#         Q2center = row['Q2center']
        nu = row['nu']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2center)+'\n'+str(nu)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. Q2center, nu:',Q2center,nu)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_q2center_nu']=RL_center
        df.loc[index,'RT_fortran_q2center_nu']=RT_center


        # calculate data
        Q2eff = row['Q2eff']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_nu', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2eff)+'\n'+str(nu)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. Q2eff, nu:',Q2eff,nu)
            data_errors.append(index)
            continue

        RT_data = data_output[4]
        RL_data = data_output[5]

        df.loc[index,'RL_fortran_q2data_nu']=RL_data
        df.loc[index,'RT_fortran_q2data_nu']=RT_data
        print('index done:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)



index done: 135 RLRT_center: 0.0736 0.0321 RLRT_data: 0.0605 0.0299
index done: 136 RLRT_center: 0.0761 0.0323 RLRT_data: 0.0639 0.0303
index done: 137 RLRT_center: 0.0786 0.0324 RLRT_data: 0.0675 0.0308
index done: 138 RLRT_center: 0.0811 0.0326 RLRT_data: 0.0713 0.0312
index done: 355 RLRT_center: 0.0838 0.0328 RLRT_data: 0.0753 0.0316
index done: 356 RLRT_center: 0.0864 0.033 RLRT_data: 0.0797 0.0321
index done: 357 RLRT_center: 0.0892 0.0331 RLRT_data: 0.0843 0.0325
index done: 358 RLRT_center: 0.092 0.0333 RLRT_data: 0.0893 0.033
index done: 359 RLRT_center: 0.0948 0.0335 RLRT_data: 0.0946 0.0334
index done: 33 RLRT_center: 0.00759 0.0178 RLRT_data: 0.00659 0.0165
index done: 34 RLRT_center: 0.00811 0.0179 RLRT_data: 0.00729 0.0169
index done: 35 RLRT_center: 0.00865 0.018 RLRT_data: 0.00807 0.0173
index done: 36 RLRT_center: 0.00921 0.0182 RLRT_data: 0.00893 0.0179
index done: 112 RLRT_center: 0.023 0.024 RLRT_data: 0.0194 0.0215
index done: 113 RLRT_center: 0.0237 0.0242 RLRT_da

index done: 13 RLRT_center: 0.00215 0.00955 RLRT_data: 0.00196 0.00903
index done: 14 RLRT_center: 0.00208 0.00938 RLRT_data: 0.00193 0.00889
index done: 15 RLRT_center: 0.00202 0.00931 RLRT_data: 0.00188 0.00879
index done: 16 RLRT_center: 0.00198 0.00935 RLRT_data: 0.00186 0.00882
index done: 17 RLRT_center: 0.00193 0.0097 RLRT_data: 0.00188 0.00935
index done: 18 RLRT_center: 0.00195 0.00998 RLRT_data: 0.00194 0.00987
index done: 19 RLRT_center: 0.00199 0.0103 RLRT_data: 0.00201 0.0105
index done: 20 RLRT_center: 0.00204 0.0106 RLRT_data: 0.00213 0.0113
index done: 21 RLRT_center: 0.00207 0.011 RLRT_data: 0.00225 0.0121
index done: 22 RLRT_center: 0.00214 0.0113 RLRT_data: 0.00241 0.0129
index done: 53 RLRT_center: 0.00229 0.0119 RLRT_data: 0.00149 0.00793
index done: 54 RLRT_center: 0.00235 0.0121 RLRT_data: 0.00159 0.0084
index done: 55 RLRT_center: 0.00242 0.0123 RLRT_data: 0.0017 0.00893
index done: 56 RLRT_center: 0.00251 0.0124 RLRT_data: 0.00185 0.00949
index done: 57 RLRT_ce

index done: 840 RLRT_center: 0.00275 0.0127 RLRT_data: 0.00182 0.00916
index done: 841 RLRT_center: 0.00288 0.0127 RLRT_data: 0.00194 0.00945
index done: 842 RLRT_center: 0.00288 0.0127 RLRT_data: 0.00194 0.00945
index done: 843 RLRT_center: 0.00288 0.0127 RLRT_data: 0.00194 0.00945
index done: 844 RLRT_center: 0.00288 0.0127 RLRT_data: 0.00194 0.00945
index done: 845 RLRT_center: 0.00302 0.0126 RLRT_data: 0.00206 0.00969
index done: 846 RLRT_center: 0.00317 0.0126 RLRT_data: 0.0022 0.00988
index done: 847 RLRT_center: 0.00334 0.0125 RLRT_data: 0.00234 0.01
index done: 848 RLRT_center: 0.00352 0.0124 RLRT_data: 0.00249 0.0102
index done: 849 RLRT_center: 0.00373 0.0124 RLRT_data: 0.00265 0.0102
index done: 850 RLRT_center: 0.00396 0.0123 RLRT_data: 0.00283 0.0103
index done: 851 RLRT_center: 0.00423 0.0122 RLRT_data: 0.00302 0.0104
index done: 852 RLRT_center: 0.00451 0.0122 RLRT_data: 0.00324 0.0104
index done: 853 RLRT_center: 0.00483 0.0122 RLRT_data: 0.00348 0.0105
index done: 854 

index done: 171 RLRT_center: 0.00142 0.00693 RLRT_data: 0.000791 0.00463
index done: 172 RLRT_center: 0.00149 0.00703 RLRT_data: 0.000846 0.0048
index done: 173 RLRT_center: 0.00156 0.00713 RLRT_data: 0.000909 0.00499
index done: 174 RLRT_center: 0.00163 0.00722 RLRT_data: 0.000979 0.00517
index done: 175 RLRT_center: 0.00171 0.00729 RLRT_data: 0.00106 0.00537
index done: 176 RLRT_center: 0.0018 0.00737 RLRT_data: 0.00115 0.00556
index done: 177 RLRT_center: 0.00189 0.00743 RLRT_data: 0.00124 0.00576
index done: 178 RLRT_center: 0.00198 0.00749 RLRT_data: 0.00135 0.00596
index done: 179 RLRT_center: 0.00208 0.00755 RLRT_data: 0.00147 0.00615
index done: 180 RLRT_center: 0.00219 0.00761 RLRT_data: 0.00159 0.00634
index done: 181 RLRT_center: 0.0023 0.00767 RLRT_data: 0.00173 0.00653
index done: 182 RLRT_center: 0.00242 0.00772 RLRT_data: 0.00188 0.00672
index done: 183 RLRT_center: 0.00255 0.00778 RLRT_data: 0.00205 0.0069
index done: 184 RLRT_center: 0.00268 0.00785 RLRT_data: 0.00222 

index done: 600 RLRT_center: 0.0147 0.0142 RLRT_data: 0.0181 0.016
index done: 601 RLRT_center: 0.015 0.0143 RLRT_data: 0.0188 0.0162
index done: 602 RLRT_center: 0.0153 0.0145 RLRT_data: 0.0195 0.0165
index done: 603 RLRT_center: 0.0161 0.0147 RLRT_data: 0.0211 0.017
index done: 604 RLRT_center: 0.0164 0.0148 RLRT_data: 0.0219 0.0173
index done: 605 RLRT_center: 0.0168 0.0149 RLRT_data: 0.0228 0.0176
index done: 606 RLRT_center: 0.0172 0.015 RLRT_data: 0.0237 0.0178
index done: 694 RLRT_center: 0.0232 0.0168 RLRT_data: 0.0145 0.0129
index done: 695 RLRT_center: 0.0236 0.0169 RLRT_data: 0.015 0.0132
index done: 696 RLRT_center: 0.0239 0.017 RLRT_data: 0.0156 0.0135
index done: 697 RLRT_center: 0.0243 0.0171 RLRT_data: 0.0162 0.0137
index done: 698 RLRT_center: 0.0247 0.0172 RLRT_data: 0.0168 0.014
index done: 806 RLRT_center: 3.71e-06 7.64e-05 RLRT_data: 1e-05 0.000159
index done: 807 RLRT_center: 6.54e-06 0.000123 RLRT_data: 1.89e-05 0.000265
index done: 808 RLRT_center: 1.15e-05 0.00

...

index done: 1175 RLRT_center: 0.0133 0.0119 RLRT_data: 0.0145 0.0126
index done: 1176 RLRT_center: 0.0135 0.012 RLRT_data: 0.0149 0.0128
index done: 1177 RLRT_center: 0.0137 0.0121 RLRT_data: 0.0154 0.013
index done: 1178 RLRT_center: 0.0139 0.0122 RLRT_data: 0.0158 0.0132
index done: 1179 RLRT_center: 0.0141 0.0123 RLRT_data: 0.0163 0.0134
index done: 240 RLRT_center: 0.000125 0.00101 RLRT_data: 4.67e-05 0.000441
index done: 241 RLRT_center: 0.000131 0.00105 RLRT_data: 5.48e-05 0.000503
index done: 242 RLRT_center: 0.000137 0.00109 RLRT_data: 6.34e-05 0.000568
index done: 243 RLRT_center: 0.000142 0.00112 RLRT_data: 7.25e-05 0.000635
index done: 244 RLRT_center: 0.000148 0.00116 RLRT_data: 8.2e-05 0.000704
index done: 245 RLRT_center: 0.000154 0.00119 RLRT_data: 9.2e-05 0.000774
index done: 246 RLRT_center: 0.000165 0.00126 RLRT_data: 0.000113 0.000918
index done: 247 RLRT_center: 0.00017 0.00129 RLRT_data: 0.000125 0.000992
index done: 248 RLRT_center: 0.000176 0.00132 RLRT_data: 0.0

index done: 782 RLRT_center: 0.00477 0.00644 RLRT_data: 0.00343 0.00518
index done: 783 RLRT_center: 0.00485 0.00649 RLRT_data: 0.00361 0.00534
index done: 784 RLRT_center: 0.00493 0.00654 RLRT_data: 0.00379 0.00551
index done: 785 RLRT_center: 0.005 0.00659 RLRT_data: 0.00398 0.00568
index done: 786 RLRT_center: 0.00508 0.00665 RLRT_data: 0.00418 0.00585
index done: 787 RLRT_center: 0.00516 0.0067 RLRT_data: 0.00438 0.00602
index done: 788 RLRT_center: 0.00523 0.00675 RLRT_data: 0.00459 0.0062
index done: 789 RLRT_center: 0.00531 0.00681 RLRT_data: 0.00481 0.00638
index done: 790 RLRT_center: 0.00539 0.00686 RLRT_data: 0.00503 0.00656
index done: 791 RLRT_center: 0.00547 0.00691 RLRT_data: 0.00527 0.00674
index done: 792 RLRT_center: 0.00555 0.00696 RLRT_data: 0.00551 0.00693
index done: 793 RLRT_center: 0.00562 0.00702 RLRT_data: 0.00576 0.00712
index done: 794 RLRT_center: 0.0057 0.00707 RLRT_data: 0.00601 0.00732
index done: 795 RLRT_center: 0.00586 0.00718 RLRT_data: 0.00655 0.007

index done: 1292 RLRT_center: 0.0073 0.00812 RLRT_data: 0.00783 0.00849
index done: 1293 RLRT_center: 0.00741 0.00819 RLRT_data: 0.00808 0.00867
index done: 1294 RLRT_center: 0.00751 0.00826 RLRT_data: 0.00835 0.00884
index done: 1295 RLRT_center: 0.00761 0.00833 RLRT_data: 0.00861 0.00901
index done: 1296 RLRT_center: 0.00772 0.00839 RLRT_data: 0.00889 0.00919
index done: 1297 RLRT_center: 0.00782 0.00846 RLRT_data: 0.00917 0.00936
index done: 1298 RLRT_center: 0.00793 0.00853 RLRT_data: 0.00946 0.00954
index done: 1299 RLRT_center: 0.00803 0.00859 RLRT_data: 0.00976 0.00972
index done: 1300 RLRT_center: 0.00814 0.00866 RLRT_data: 0.0101 0.00991
index done: 1301 RLRT_center: 0.00824 0.00873 RLRT_data: 0.0104 0.0101
index done: 229 RLRT_center: 1.56e-06 3.56e-05 RLRT_data: 2.75e-06 5.22e-05
index done: 230 RLRT_center: 1.9e-06 4e-05 RLRT_data: 3.8e-06 6.48e-05
index done: 231 RLRT_center: 2.3e-06 4.51e-05 RLRT_data: 5.21e-06 8.08e-05
index done: 232 RLRT_center: 2.78e-06 5.09e-05 RLRT_

index done: 1062 RLRT_center: 7.68e-06 0.000103 RLRT_data: 2.09e-05 0.000228
index done: 1063 RLRT_center: 9.44e-06 0.00012 RLRT_data: 2.59e-05 0.000271
index done: 1064 RLRT_center: 1.15e-05 0.000139 RLRT_data: 3.15e-05 0.000317
index done: 1065 RLRT_center: 1.38e-05 0.00016 RLRT_data: 3.75e-05 0.000366
index done: 1066 RLRT_center: 1.64e-05 0.000183 RLRT_data: 4.4e-05 0.000417
index done: 1067 RLRT_center: 1.92e-05 0.000208 RLRT_data: 5.08e-05 0.000469
index done: 1068 RLRT_center: 2.22e-05 0.000234 RLRT_data: 5.79e-05 0.000522
index done: 1069 RLRT_center: 2.55e-05 0.00026 RLRT_data: 6.52e-05 0.000575
index done: 1070 RLRT_center: 2.88e-05 0.000288 RLRT_data: 7.28e-05 0.000629
index done: 1071 RLRT_center: 3.23e-05 0.000316 RLRT_data: 8.06e-05 0.000684
index done: 1072 RLRT_center: 3.59e-05 0.000343 RLRT_data: 8.87e-05 0.000738
index done: 1223 RLRT_center: 0.000523 0.00167 RLRT_data: 0.000278 0.00107
index done: 1224 RLRT_center: 0.000547 0.00171 RLRT_data: 0.000301 0.00113
index d

index done: 1180 RLRT_center: 4.71e-07 5.56e-06 RLRT_data: 2.11e-06 3.12e-05


In [12]:
# compute Fortran fit RL, RT for the dataset (q2 center, q2 data) analysis in Ex
df['RL_fortran_q2center_ex']=0
df['RT_fortran_q2center_ex']=0
df['RL_fortran_q2data_ex']=0
df['RT_fortran_q2data_ex']=0

center_errors = []
data_errors = []
fortran_folder = "Eric/response/"
# for index, row in df.iterrows():
for Q2center in Q2centers:
    picked = df.loc[df['Q2center']==Q2center]
    for index, row in picked.iterrows():
        # calculate center
#         Q2center = row['Q2center']
        Ex = row['Ex']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2center)+'\n'+str(Ex)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. Q2center, Ex:',Q2center,Ex)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_q2center_ex']=RL_center
        df.loc[index,'RT_fortran_q2center_ex']=RT_center


        # calculate data
        Q2eff = row['Q2eff']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_ex', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2eff)+'\n'+str(Ex)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. Q2eff, Ex:',Q2eff,Ex)
            data_errors.append(index)
            continue

        RT_data = data_output[4]
        RL_data = data_output[5]

        df.loc[index,'RL_fortran_q2data_ex']=RL_data
        df.loc[index,'RT_fortran_q2data_ex']=RT_data
        print('index done:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)



index done: 135 RLRT_center: 0.0708 0.0319 RLRT_data: 0.0585 0.0298
index done: 136 RLRT_center: 0.0732 0.0321 RLRT_data: 0.0617 0.0302
index done: 137 RLRT_center: 0.0756 0.0323 RLRT_data: 0.0652 0.0306
index done: 138 RLRT_center: 0.0781 0.0324 RLRT_data: 0.0689 0.031
index done: 355 RLRT_center: 0.0807 0.0326 RLRT_data: 0.0727 0.0314
index done: 356 RLRT_center: 0.0832 0.0328 RLRT_data: 0.0769 0.0319
index done: 357 RLRT_center: 0.0859 0.0329 RLRT_data: 0.0813 0.0323
index done: 358 RLRT_center: 0.0886 0.0331 RLRT_data: 0.0861 0.0328
index done: 359 RLRT_center: 0.0914 0.0333 RLRT_data: 0.0912 0.0332
index done: 33 RLRT_center: 0.00682 0.0177 RLRT_data: 0.00601 0.0166
index done: 34 RLRT_center: 0.00729 0.0177 RLRT_data: 0.00662 0.0168
index done: 35 RLRT_center: 0.00779 0.0178 RLRT_data: 0.00731 0.0172
index done: 36 RLRT_center: 0.00832 0.0179 RLRT_data: 0.00808 0.0176
index done: 112 RLRT_center: 0.0223 0.0239 RLRT_data: 0.019 0.0214
index done: 113 RLRT_center: 0.023 0.0241 RLRT

index done: 919 RLRT_center: 0.0476 0.025 RLRT_data: 0.0408 0.0233
index done: 8 RLRT_center: 0.00259 0.0114 RLRT_data: 0.00111 0.00583
index done: 9 RLRT_center: 0.00262 0.0112 RLRT_data: 0.00174 0.0084
index done: 10 RLRT_center: 0.00254 0.0109 RLRT_data: 0.00189 0.0089
index done: 11 RLRT_center: 0.00245 0.0105 RLRT_data: 0.00197 0.00918
index done: 12 RLRT_center: 0.00234 0.0101 RLRT_data: 0.00201 0.00927
index done: 13 RLRT_center: 0.00224 0.00979 RLRT_data: 0.00201 0.00921
index done: 14 RLRT_center: 0.00215 0.00953 RLRT_data: 0.00198 0.00907
index done: 15 RLRT_center: 0.00207 0.00937 RLRT_data: 0.00194 0.00894
index done: 16 RLRT_center: 0.00201 0.00931 RLRT_data: 0.0019 0.00888
index done: 17 RLRT_center: 0.00195 0.0095 RLRT_data: 0.0019 0.00921
index done: 18 RLRT_center: 0.00193 0.00971 RLRT_data: 0.00192 0.00963
index done: 19 RLRT_center: 0.00195 0.00999 RLRT_data: 0.002 0.0102
index done: 20 RLRT_center: 0.00199 0.0103 RLRT_data: 0.00208 0.0109
index done: 21 RLRT_center:

index done: 831 RLRT_center: 0.00235 0.0121 RLRT_data: 0.00153 0.00811
index done: 832 RLRT_center: 0.00246 0.0124 RLRT_data: 0.00161 0.00845
index done: 833 RLRT_center: 0.00258 0.0125 RLRT_data: 0.0017 0.00879
index done: 834 RLRT_center: 0.00258 0.0125 RLRT_data: 0.0017 0.00879
index done: 835 RLRT_center: 0.00258 0.0125 RLRT_data: 0.0017 0.00879
index done: 836 RLRT_center: 0.00258 0.0125 RLRT_data: 0.0017 0.00879
index done: 837 RLRT_center: 0.00268 0.0126 RLRT_data: 0.00181 0.00911
index done: 838 RLRT_center: 0.00268 0.0126 RLRT_data: 0.00181 0.00911
index done: 839 RLRT_center: 0.00268 0.0126 RLRT_data: 0.00181 0.00911
index done: 840 RLRT_center: 0.00268 0.0126 RLRT_data: 0.00181 0.00911
index done: 841 RLRT_center: 0.00281 0.0127 RLRT_data: 0.00192 0.0094
index done: 842 RLRT_center: 0.00281 0.0127 RLRT_data: 0.00192 0.0094
index done: 843 RLRT_center: 0.00281 0.0127 RLRT_data: 0.00192 0.0094
index done: 844 RLRT_center: 0.00281 0.0127 RLRT_data: 0.00192 0.0094
index done: 84

index done: 80 RLRT_center: 0.00103 0.00586 RLRT_data: 0.000895 0.00525
index done: 81 RLRT_center: 0.00105 0.00597 RLRT_data: 0.000984 0.00568
index done: 82 RLRT_center: 0.00107 0.00604 RLRT_data: 0.00104 0.00594
index done: 83 RLRT_center: 0.00108 0.00611 RLRT_data: 0.00112 0.00624
index done: 84 RLRT_center: 0.00111 0.00619 RLRT_data: 0.0012 0.00658
index done: 85 RLRT_center: 0.00113 0.00628 RLRT_data: 0.0013 0.00696
index done: 86 RLRT_center: 0.00116 0.00637 RLRT_data: 0.00141 0.00736
index done: 87 RLRT_center: 0.00119 0.00646 RLRT_data: 0.00154 0.00779
index done: 165 RLRT_center: 0.00106 0.00602 RLRT_data: 0.000599 0.00378
index done: 166 RLRT_center: 0.00109 0.00613 RLRT_data: 0.000618 0.0039
index done: 167 RLRT_center: 0.00112 0.00624 RLRT_data: 0.000637 0.00401
index done: 168 RLRT_center: 0.00116 0.00636 RLRT_data: 0.00066 0.00413
index done: 169 RLRT_center: 0.00121 0.00648 RLRT_data: 0.000687 0.00426
index done: 170 RLRT_center: 0.00126 0.00661 RLRT_data: 0.00072 0.004

index done: 585 RLRT_center: 0.00925 0.012 RLRT_data: 0.00881 0.0117
index done: 586 RLRT_center: 0.00952 0.0122 RLRT_data: 0.00921 0.0119
index done: 587 RLRT_center: 0.00979 0.0123 RLRT_data: 0.00963 0.0122
index done: 588 RLRT_center: 0.0101 0.0124 RLRT_data: 0.0101 0.0124
index done: 589 RLRT_center: 0.0106 0.0127 RLRT_data: 0.011 0.013
index done: 590 RLRT_center: 0.0109 0.0128 RLRT_data: 0.0115 0.0132
index done: 591 RLRT_center: 0.0112 0.013 RLRT_data: 0.012 0.0135
index done: 592 RLRT_center: 0.0115 0.0131 RLRT_data: 0.0125 0.0137
index done: 593 RLRT_center: 0.0118 0.0132 RLRT_data: 0.013 0.014
index done: 594 RLRT_center: 0.0121 0.0133 RLRT_data: 0.0136 0.0142
index done: 595 RLRT_center: 0.0125 0.0134 RLRT_data: 0.0141 0.0145
index done: 596 RLRT_center: 0.0128 0.0136 RLRT_data: 0.0147 0.0147
index done: 597 RLRT_center: 0.0131 0.0137 RLRT_data: 0.0153 0.015
index done: 598 RLRT_center: 0.0134 0.0138 RLRT_data: 0.016 0.0152
index done: 599 RLRT_center: 0.0138 0.0139 RLRT_dat

...

index done: 1156 RLRT_center: 0.00952 0.0102 RLRT_data: 0.00808 0.00921
index done: 1157 RLRT_center: 0.00971 0.0103 RLRT_data: 0.00838 0.00939
index done: 1158 RLRT_center: 0.0099 0.0104 RLRT_data: 0.00869 0.00957
index done: 1159 RLRT_center: 0.0101 0.0105 RLRT_data: 0.009 0.00975
index done: 1160 RLRT_center: 0.0101 0.0105 RLRT_data: 0.009 0.00975
index done: 1161 RLRT_center: 0.0101 0.0105 RLRT_data: 0.009 0.00975
index done: 1162 RLRT_center: 0.0101 0.0105 RLRT_data: 0.009 0.00975
index done: 1163 RLRT_center: 0.0103 0.0105 RLRT_data: 0.00932 0.00993
index done: 1164 RLRT_center: 0.0105 0.0106 RLRT_data: 0.00964 0.0101
index done: 1165 RLRT_center: 0.0107 0.0107 RLRT_data: 0.00997 0.0103
index done: 1166 RLRT_center: 0.0109 0.0108 RLRT_data: 0.0103 0.0105
index done: 1167 RLRT_center: 0.0111 0.0109 RLRT_data: 0.0107 0.0107
index done: 1168 RLRT_center: 0.0113 0.011 RLRT_data: 0.011 0.0109
index done: 1169 RLRT_center: 0.0114 0.0111 RLRT_data: 0.0114 0.0111
index done: 1170 RLRT_ce

index done: 732 RLRT_center: 0.00259 0.00469 RLRT_data: 0.00187 0.00376
index done: 733 RLRT_center: 0.00266 0.00475 RLRT_data: 0.00199 0.0039
index done: 734 RLRT_center: 0.00272 0.00481 RLRT_data: 0.00211 0.00405
index done: 735 RLRT_center: 0.00286 0.00493 RLRT_data: 0.00237 0.00434
index done: 736 RLRT_center: 0.00292 0.005 RLRT_data: 0.0025 0.0045
index done: 737 RLRT_center: 0.00299 0.00506 RLRT_data: 0.00265 0.00465
index done: 738 RLRT_center: 0.00306 0.00512 RLRT_data: 0.00279 0.00481
index done: 739 RLRT_center: 0.00313 0.00518 RLRT_data: 0.00295 0.00498
index done: 740 RLRT_center: 0.0032 0.00524 RLRT_data: 0.00311 0.00514
index done: 741 RLRT_center: 0.00327 0.0053 RLRT_data: 0.00328 0.00531
index done: 742 RLRT_center: 0.00334 0.00536 RLRT_data: 0.00345 0.00548
index done: 743 RLRT_center: 0.00342 0.00542 RLRT_data: 0.00364 0.00565
index done: 744 RLRT_center: 0.00349 0.00548 RLRT_data: 0.00383 0.00582
index done: 745 RLRT_center: 0.00356 0.00553 RLRT_data: 0.00402 0.006
i

index done: 1274 RLRT_center: 0.00512 0.00668 RLRT_data: 0.00407 0.00573
index done: 1275 RLRT_center: 0.00522 0.00675 RLRT_data: 0.00424 0.00587
index done: 1276 RLRT_center: 0.00532 0.00681 RLRT_data: 0.00441 0.00602
index done: 1277 RLRT_center: 0.00542 0.00688 RLRT_data: 0.00459 0.00617
index done: 1278 RLRT_center: 0.00552 0.00695 RLRT_data: 0.00477 0.00631
index done: 1279 RLRT_center: 0.00562 0.00702 RLRT_data: 0.00496 0.00646
index done: 1280 RLRT_center: 0.00572 0.00708 RLRT_data: 0.00516 0.00661
index done: 1281 RLRT_center: 0.00582 0.00715 RLRT_data: 0.00535 0.00677
index done: 1282 RLRT_center: 0.00582 0.00715 RLRT_data: 0.00535 0.00677
index done: 1283 RLRT_center: 0.00582 0.00715 RLRT_data: 0.00535 0.00677
index done: 1284 RLRT_center: 0.00582 0.00715 RLRT_data: 0.00535 0.00677
index done: 1285 RLRT_center: 0.00592 0.00722 RLRT_data: 0.00556 0.00692
index done: 1286 RLRT_center: 0.00603 0.00729 RLRT_data: 0.00576 0.00708
index done: 1287 RLRT_center: 0.00613 0.00735 RLRT_

index done: 778 RLRT_center: 0.00131 0.00281 RLRT_data: 0.00191 0.00366
index done: 779 RLRT_center: 0.00135 0.00285 RLRT_data: 0.00203 0.0038
index done: 780 RLRT_center: 0.00138 0.00289 RLRT_data: 0.00214 0.00394
index done: 1052 RLRT_center: 5.23e-07 5.79e-06 RLRT_data: 8.02e-07 8.52e-06
index done: 1053 RLRT_center: 6.85e-07 7.43e-06 RLRT_data: 1.13e-06 3.06e-05
index done: 1054 RLRT_center: 8.95e-07 2.59e-05 RLRT_data: 1.57e-06 3.72e-05
index done: 1055 RLRT_center: 1.17e-06 3e-05 RLRT_data: 2.19e-06 4.55e-05
index done: 1056 RLRT_center: 1.52e-06 3.49e-05 RLRT_data: 3.04e-06 5.61e-05
index done: 1057 RLRT_center: 1.97e-06 4.09e-05 RLRT_data: 4.18e-06 6.95e-05
index done: 1058 RLRT_center: 2.54e-06 4.8e-05 RLRT_data: 5.69e-06 8.61e-05
index done: 1059 RLRT_center: 3.25e-06 5.65e-05 RLRT_data: 7.66e-06 0.000107
index done: 1060 RLRT_center: 4.15e-06 6.65e-05 RLRT_data: 1.02e-05 0.000131
index done: 1061 RLRT_center: 5.23e-06 7.82e-05 RLRT_data: 1.33e-05 0.000161
index done: 1062 RL

index done: 1206 RLRT_center: 5.59e-05 0.000398 RLRT_data: 6.74e-05 0.000462
index done: 1207 RLRT_center: 5.91e-05 0.000412 RLRT_data: 7.36e-05 0.00049
index done: 1208 RLRT_center: 6.26e-05 0.000426 RLRT_data: 8.02e-05 0.000518
index done: 1209 RLRT_center: 6.26e-05 0.000426 RLRT_data: 8.02e-05 0.000518
index done: 1210 RLRT_center: 6.26e-05 0.000426 RLRT_data: 8.02e-05 0.000518
index done: 1211 RLRT_center: 6.26e-05 0.000426 RLRT_data: 8.02e-05 0.000518
index done: 1212 RLRT_center: 6.62e-05 0.000441 RLRT_data: 8.75e-05 0.000548
index done: 1213 RLRT_center: 7.01e-05 0.000456 RLRT_data: 9.54e-05 0.000579
index done: 1214 RLRT_center: 7.41e-05 0.000471 RLRT_data: 0.000104 0.00061
index done: 1215 RLRT_center: 7.83e-05 0.000486 RLRT_data: 0.000113 0.000643
index done: 1216 RLRT_center: 8.28e-05 0.000501 RLRT_data: 0.000123 0.000677
index done: 1217 RLRT_center: 8.75e-05 0.000517 RLRT_data: 0.000134 0.000712
index done: 1218 RLRT_center: 9.25e-05 0.000533 RLRT_data: 0.000147 0.000749
i

In [13]:
# compute Fortran fit RL, RT for the dataset (qv center, qv data) analysis in W2
df['RL_fortran_qvcenter_w2']=0
df['RT_fortran_qvcenter_w2']=0
df['RL_fortran_qvdata_w2']=0
df['RT_fortran_qvdata_w2']=0

center_errors = []
data_errors = []
fortran_folder = "Eric/response/"
# for index, row in df.iterrows():
for qvcenter in qvcenters:
    picked = df.loc[df['qvcenter']==qvcenter]
    for index, row in picked.iterrows():
        # calculate center
        W2 = row['W2']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qvcenter)+'\n'+str(W2)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. qvcenter, W2:',qvcenter,W2)
            center_errors.append(index)
            continue
    #     qv,q2,ex,nu,RTTOT,RLTOT,RTQE,RLQE,RTIE,RLIE,RTE,RLE,RTNS,RLNS
        ex_center = center_output[2]
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_qvcenter_w2']=RL_center
        df.loc[index,'RT_fortran_qvcenter_w2']=RT_center



        # calculate data
        qv = row['q3momt']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_qv_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(qv)+'\n'+str(W2)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. qv, W2:',qv,W2)
            data_errors.append(index)
            continue
        ex_data = data_output[2]
        RT_data = data_output[4]
        RL_data = data_output[5]

        print('index:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)
    #     print('index:',index,'Ex:',Ex,'ex_data:',ex_data,'Ex-ex_data:',Ex-ex_data )
        df.loc[index,'RL_fortran_qvdata_w2']=RL_data
        df.loc[index,'RT_fortran_qvdata_w2']=RT_data




index: 0 RLRT_center: 2.08e-05 0.00023 RLRT_data: 3.57e-05 0.000428
index: 1 RLRT_center: 4.3e-05 0.000443 RLRT_data: 6.48e-05 0.000659
index: 2 RLRT_center: 8.68e-05 0.000788 RLRT_data: 0.000115 0.001
index: 3 RLRT_center: 0.000171 0.00134 RLRT_data: 0.000199 0.0015
index: 4 RLRT_center: 0.000322 0.00217 RLRT_data: 0.000327 0.00219
index: 5 RLRT_center: 0.000563 0.00335 RLRT_data: 0.000502 0.00306
index: 6 RLRT_center: 0.000892 0.00482 RLRT_data: 0.000715 0.00406
index: 7 RLRT_center: 0.00128 0.00646 RLRT_data: 0.000951 0.00512
index: 8 RLRT_center: 0.00168 0.00807 RLRT_data: 0.00119 0.00616
index: 9 RLRT_center: 0.00264 0.0115 RLRT_data: 0.00179 0.00854
index: 10 RLRT_center: 0.0028 0.0119 RLRT_data: 0.0019 0.00895
index: 11 RLRT_center: 0.00286 0.0119 RLRT_data: 0.00197 0.00914
index: 12 RLRT_center: 0.00283 0.0116 RLRT_data: 0.00198 0.00914
index: 13 RLRT_center: 0.00274 0.0112 RLRT_data: 0.00196 0.00903
index: 14 RLRT_center: 0.00263 0.0107 RLRT_data: 0.00193 0.00889
index: 15 RLR

index: 103 RLRT_center: 0.0189 0.0278 RLRT_data: 0.0126 0.0174
index: 104 RLRT_center: 0.021 0.0296 RLRT_data: 0.0133 0.0179
index: 105 RLRT_center: 0.0241 0.0314 RLRT_data: 0.014 0.0184
index: 106 RLRT_center: 0.0308 0.0334 RLRT_data: 0.0147 0.0188
index: 107 RLRT_center: 0.0483 0.0353 RLRT_data: 0.0154 0.0193
index: 108 RLRT_center: 0.1 0.0372 RLRT_data: 0.0162 0.0198
index: 109 RLRT_center: 0.27 0.0391 RLRT_data: 0.017 0.0202
index: 139 RLRT_center: 8.71e-08 2.71e-06 RLRT_data: 3.49e-07 2.04e-05
index: 140 RLRT_center: 1.97e-07 6.81e-06 RLRT_data: 5.51e-07 2.74e-05
index: 141 RLRT_center: 3.47e-07 1.24e-05 RLRT_data: 8.71e-07 3.66e-05
index: 142 RLRT_center: 6.04e-07 2.09e-05 RLRT_data: 1.38e-06 4.88e-05
index: 143 RLRT_center: 1.05e-06 3.36e-05 RLRT_data: 2.2e-06 6.49e-05
index: 144 RLRT_center: 1.8e-06 5.19e-05 RLRT_data: 3.49e-06 8.62e-05
index: 145 RLRT_center: 3.12e-06 7.83e-05 RLRT_data: 5.55e-06 0.000115
index: 146 RLRT_center: 5.4e-06 0.000116 RLRT_data: 8.82e-06 0.000154
in

index: 828 RLRT_center: 0.00136 0.00722 RLRT_data: 0.00141 0.00741
index: 829 RLRT_center: 0.00142 0.00756 RLRT_data: 0.00143 0.00761
index: 830 RLRT_center: 0.0015 0.00799 RLRT_data: 0.00148 0.00786
index: 831 RLRT_center: 0.0016 0.00849 RLRT_data: 0.00154 0.00817
index: 832 RLRT_center: 0.00174 0.00906 RLRT_data: 0.00162 0.00851
index: 833 RLRT_center: 0.00188 0.00966 RLRT_data: 0.00172 0.00885
index: 834 RLRT_center: 0.00188 0.00966 RLRT_data: 0.00172 0.00885
index: 835 RLRT_center: 0.00188 0.00966 RLRT_data: 0.00172 0.00885
index: 836 RLRT_center: 0.00188 0.00966 RLRT_data: 0.00172 0.00885
index: 837 RLRT_center: 0.00204 0.0103 RLRT_data: 0.00182 0.00916
index: 838 RLRT_center: 0.00204 0.0103 RLRT_data: 0.00182 0.00916
index: 839 RLRT_center: 0.00204 0.0103 RLRT_data: 0.00182 0.00916
index: 840 RLRT_center: 0.00204 0.0103 RLRT_data: 0.00182 0.00916
index: 841 RLRT_center: 0.00221 0.0109 RLRT_data: 0.00194 0.00945
index: 842 RLRT_center: 0.00221 0.0109 RLRT_data: 0.00194 0.00945
ind

index: 240 RLRT_center: 5.2e-05 0.000486 RLRT_data: 4.67e-05 0.000441
index: 241 RLRT_center: 6.16e-05 0.000559 RLRT_data: 5.48e-05 0.000503
index: 242 RLRT_center: 7.2e-05 0.000637 RLRT_data: 6.34e-05 0.000568
index: 243 RLRT_center: 8.3e-05 0.000717 RLRT_data: 7.25e-05 0.000635
index: 244 RLRT_center: 9.46e-05 0.0008 RLRT_data: 8.2e-05 0.000704
index: 245 RLRT_center: 0.000107 0.000885 RLRT_data: 9.2e-05 0.000774
index: 246 RLRT_center: 0.000133 0.00106 RLRT_data: 0.000113 0.000918
index: 247 RLRT_center: 0.000147 0.00115 RLRT_data: 0.000125 0.000992
index: 248 RLRT_center: 0.000161 0.00125 RLRT_data: 0.000136 0.00107
index: 249 RLRT_center: 0.000177 0.00134 RLRT_data: 0.000149 0.00114
index: 250 RLRT_center: 0.000192 0.00143 RLRT_data: 0.000162 0.00122
index: 251 RLRT_center: 0.000209 0.00153 RLRT_data: 0.000175 0.00129
index: 252 RLRT_center: 0.000226 0.00163 RLRT_data: 0.00019 0.00137
index: 253 RLRT_center: 0.000245 0.00173 RLRT_data: 0.000205 0.00145
index: 254 RLRT_center: 0.00

index: 879 RLRT_center: 0.00833 0.0118 RLRT_data: 0.0117 0.0151
index: 880 RLRT_center: 0.00897 0.0123 RLRT_data: 0.0122 0.0154
index: 881 RLRT_center: 0.00966 0.0129 RLRT_data: 0.0127 0.0157
index: 882 RLRT_center: 0.0104 0.0134 RLRT_data: 0.0132 0.016
index: 883 RLRT_center: 0.0112 0.014 RLRT_data: 0.0138 0.0163
index: 884 RLRT_center: 0.0121 0.0146 RLRT_data: 0.0143 0.0166
index: 885 RLRT_center: 0.013 0.0152 RLRT_data: 0.0149 0.0168
index: 886 RLRT_center: 0.014 0.0159 RLRT_data: 0.0155 0.0171
index: 887 RLRT_center: 0.015 0.0165 RLRT_data: 0.0161 0.0173
index: 888 RLRT_center: 0.0162 0.0172 RLRT_data: 0.0167 0.0176
index: 889 RLRT_center: 0.0175 0.0179 RLRT_data: 0.0174 0.0178
index: 890 RLRT_center: 0.0189 0.0186 RLRT_data: 0.0181 0.0181
index: 891 RLRT_center: 0.0204 0.0194 RLRT_data: 0.0188 0.0183
index: 892 RLRT_center: 0.0221 0.0202 RLRT_data: 0.0195 0.0185
index: 893 RLRT_center: 0.024 0.021 RLRT_data: 0.0202 0.0187
index: 894 RLRT_center: 0.0261 0.0218 RLRT_data: 0.021 0.01

...

index: 623 RLRT_center: 8.82e-05 0.000685 RLRT_data: 0.000116 0.00091
index: 624 RLRT_center: 9.71e-05 0.000737 RLRT_data: 0.000126 0.000969
index: 625 RLRT_center: 0.000107 0.000791 RLRT_data: 0.000136 0.00103
index: 626 RLRT_center: 0.000117 0.000847 RLRT_data: 0.000147 0.00108
index: 627 RLRT_center: 0.000129 0.000904 RLRT_data: 0.000159 0.00114
index: 628 RLRT_center: 0.000141 0.000964 RLRT_data: 0.000172 0.0012
index: 629 RLRT_center: 0.000155 0.00103 RLRT_data: 0.000186 0.00126
index: 630 RLRT_center: 0.00017 0.00109 RLRT_data: 0.000201 0.00132
index: 631 RLRT_center: 0.000186 0.00116 RLRT_data: 0.000218 0.00139
index: 632 RLRT_center: 0.000205 0.00123 RLRT_data: 0.000236 0.00145
index: 633 RLRT_center: 0.000225 0.0013 RLRT_data: 0.000257 0.00152
index: 634 RLRT_center: 0.000248 0.00138 RLRT_data: 0.00028 0.00159
index: 635 RLRT_center: 0.000273 0.00146 RLRT_data: 0.000305 0.00166
index: 636 RLRT_center: 0.000301 0.00155 RLRT_data: 0.000333 0.00173
index: 637 RLRT_center: 0.00040

index: 339 RLRT_center: 0.000521 0.00175 RLRT_data: 0.00064 0.00216
index: 340 RLRT_center: 0.000571 0.00185 RLRT_data: 0.000701 0.00228
index: 341 RLRT_center: 0.000625 0.00196 RLRT_data: 0.000767 0.0024
index: 342 RLRT_center: 0.000683 0.00208 RLRT_data: 0.000838 0.00253
index: 343 RLRT_center: 0.000746 0.0022 RLRT_data: 0.000915 0.00267
index: 344 RLRT_center: 0.000814 0.00232 RLRT_data: 0.000997 0.00281
index: 345 RLRT_center: 0.000965 0.00259 RLRT_data: 0.00118 0.00311
index: 346 RLRT_center: 0.00105 0.00273 RLRT_data: 0.00128 0.00327
index: 347 RLRT_center: 0.00114 0.00288 RLRT_data: 0.00139 0.00343
index: 348 RLRT_center: 0.00124 0.00303 RLRT_data: 0.0015 0.0036
index: 349 RLRT_center: 0.00134 0.00319 RLRT_data: 0.00163 0.00378
index: 350 RLRT_center: 0.00145 0.00336 RLRT_data: 0.00176 0.00396
index: 351 RLRT_center: 0.00157 0.00353 RLRT_data: 0.0019 0.00415
index: 352 RLRT_center: 0.0017 0.00371 RLRT_data: 0.00205 0.00435
index: 353 RLRT_center: 0.00183 0.0039 RLRT_data: 0.0022

index: 1043 RLRT_center: 0.0227 0.0171 RLRT_data: 0.0176 0.0147
index: 1044 RLRT_center: 0.0243 0.0177 RLRT_data: 0.0182 0.0149
index: 1116 RLRT_center: 0.00116 0.0029 RLRT_data: 0.00143 0.00351
index: 1117 RLRT_center: 0.00125 0.00306 RLRT_data: 0.00153 0.00365
index: 1118 RLRT_center: 0.00136 0.00322 RLRT_data: 0.00163 0.00378
index: 1119 RLRT_center: 0.00147 0.00339 RLRT_data: 0.00174 0.00392
index: 1120 RLRT_center: 0.00159 0.00356 RLRT_data: 0.00185 0.00406
index: 1121 RLRT_center: 0.00172 0.00374 RLRT_data: 0.00197 0.0042
index: 1122 RLRT_center: 0.00185 0.00393 RLRT_data: 0.00209 0.00434
index: 1123 RLRT_center: 0.002 0.00412 RLRT_data: 0.00221 0.00449
index: 1124 RLRT_center: 0.00215 0.00433 RLRT_data: 0.00234 0.00464
index: 1125 RLRT_center: 0.00232 0.00453 RLRT_data: 0.00248 0.00479
index: 1126 RLRT_center: 0.00249 0.00475 RLRT_data: 0.00262 0.00494
index: 1127 RLRT_center: 0.00268 0.00497 RLRT_data: 0.00276 0.0051
index: 1128 RLRT_center: 0.00287 0.0052 RLRT_data: 0.00291 0.

index: 755 RLRT_center: 0.000513 0.00151 RLRT_data: 0.000504 0.00149
index: 756 RLRT_center: 0.000557 0.0016 RLRT_data: 0.000543 0.00156
index: 757 RLRT_center: 0.000604 0.00169 RLRT_data: 0.000585 0.00164
index: 758 RLRT_center: 0.000655 0.00179 RLRT_data: 0.00063 0.00172
index: 759 RLRT_center: 0.000709 0.00189 RLRT_data: 0.000678 0.00181
index: 760 RLRT_center: 0.000767 0.00199 RLRT_data: 0.000728 0.0019
index: 761 RLRT_center: 0.00083 0.0021 RLRT_data: 0.000781 0.00199
index: 762 RLRT_center: 0.000896 0.00222 RLRT_data: 0.000837 0.00208
index: 763 RLRT_center: 0.000967 0.00233 RLRT_data: 0.000897 0.00218
index: 764 RLRT_center: 0.00104 0.00246 RLRT_data: 0.000959 0.00228
index: 765 RLRT_center: 0.00112 0.00258 RLRT_data: 0.00103 0.00238
index: 766 RLRT_center: 0.00121 0.00272 RLRT_data: 0.0011 0.00248
index: 767 RLRT_center: 0.0015 0.00314 RLRT_data: 0.00133 0.00282
index: 768 RLRT_center: 0.00161 0.00329 RLRT_data: 0.00141 0.00293
index: 769 RLRT_center: 0.00173 0.00345 RLRT_data:

index: 1176 RLRT_center: 0.0136 0.0121 RLRT_data: 0.0149 0.0128
index: 1177 RLRT_center: 0.0143 0.0124 RLRT_data: 0.0154 0.013
index: 1178 RLRT_center: 0.0151 0.0128 RLRT_data: 0.0158 0.0132
index: 1179 RLRT_center: 0.0159 0.0132 RLRT_data: 0.0163 0.0134
index: 1258 RLRT_center: 0.00187 0.0034 RLRT_data: 0.00225 0.00396
index: 1259 RLRT_center: 0.00199 0.00354 RLRT_data: 0.00236 0.00408
index: 1260 RLRT_center: 0.00211 0.00369 RLRT_data: 0.00248 0.00421
index: 1261 RLRT_center: 0.00225 0.00385 RLRT_data: 0.0026 0.00433
index: 1262 RLRT_center: 0.00239 0.00401 RLRT_data: 0.00273 0.00446
index: 1263 RLRT_center: 0.00254 0.00417 RLRT_data: 0.00286 0.00459
index: 1264 RLRT_center: 0.00269 0.00434 RLRT_data: 0.003 0.00472
index: 1265 RLRT_center: 0.00286 0.00452 RLRT_data: 0.00313 0.00486
index: 1266 RLRT_center: 0.00303 0.0047 RLRT_data: 0.00328 0.00499
index: 1267 RLRT_center: 0.00321 0.00488 RLRT_data: 0.00343 0.00513
index: 1268 RLRT_center: 0.0034 0.00507 RLRT_data: 0.00358 0.00526
ind

In [14]:
# compute Fortran fit RL, RT for the dataset (q2 center, q2 data) analysis in W2
df['RL_fortran_q2center_w2']=0
df['RT_fortran_q2center_w2']=0
df['RL_fortran_q2data_w2']=0
df['RT_fortran_q2data_w2']=0

center_errors = []
data_errors = []
fortran_folder = "Eric/response/"
# for index, row in df.iterrows():
for Q2center in Q2centers:
    picked = df.loc[df['Q2center']==Q2center]
    for index, row in picked.iterrows():
        # calculate center
        W2 = row['W2']
        center_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2center)+'\n'+str(W2)+'\n'
            center_output, _ = process.communicate(input_data)
        center_output = center_output.split()
        center_output = np.array(list(map(float, center_output)))
        if len(center_output)==0:
            print('ERROR index:',index,'center no result. Q2center, W2:',Q2center,W2)
            center_errors.append(index)
            continue
        RT_center = center_output[4]
        RL_center = center_output[5]
        df.loc[index,'RL_fortran_q2center_w2']=RL_center
        df.loc[index,'RT_fortran_q2center_w2']=RT_center


        # calculate data
        Q2eff = row['Q2eff']
        data_output = ''
        with subprocess.Popen(fortran_folder+'response_q2_w2', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
            input_data = str(Q2eff)+'\n'+str(W2)+'\n'
            data_output, _ = process.communicate(input_data)
        data_output = data_output.split()
        data_output = np.array(list(map(float, data_output)))
        if len(data_output)==0:
            print('ERROR index:',index,'data no result. Q2eff, W2:',Q2eff,W2)
            data_errors.append(index)
            continue

        RT_data = data_output[4]
        RL_data = data_output[5]

        df.loc[index,'RL_fortran_q2data_w2']=RL_data
        df.loc[index,'RT_fortran_q2data_w2']=RT_data
        print('index done:',index,'RLRT_center:',RL_center,RT_center,'RLRT_data:',RL_data,RT_data)



index done: 135 RLRT_center: 0.0707 0.0319 RLRT_data: 0.0605 0.0299
index done: 136 RLRT_center: 0.0735 0.0321 RLRT_data: 0.0639 0.0303
index done: 137 RLRT_center: 0.0763 0.0323 RLRT_data: 0.0675 0.0308
index done: 138 RLRT_center: 0.0792 0.0325 RLRT_data: 0.0713 0.0312
index done: 355 RLRT_center: 0.0822 0.0327 RLRT_data: 0.0753 0.0316
index done: 356 RLRT_center: 0.0852 0.0329 RLRT_data: 0.0797 0.0321
index done: 357 RLRT_center: 0.0883 0.0331 RLRT_data: 0.0843 0.0325
index done: 358 RLRT_center: 0.0915 0.0333 RLRT_data: 0.0893 0.033
index done: 359 RLRT_center: 0.0948 0.0335 RLRT_data: 0.0946 0.0334
index done: 33 RLRT_center: 0.00691 0.0177 RLRT_data: 0.00659 0.0165
index done: 34 RLRT_center: 0.00757 0.0178 RLRT_data: 0.00729 0.0169
index done: 35 RLRT_center: 0.00828 0.0179 RLRT_data: 0.00807 0.0173
index done: 36 RLRT_center: 0.00904 0.0182 RLRT_data: 0.00893 0.0179
index done: 112 RLRT_center: 0.0218 0.0237 RLRT_data: 0.0194 0.0215
index done: 113 RLRT_center: 0.0226 0.024 RLR

index done: 13 RLRT_center: 0.00239 0.0103 RLRT_data: 0.00196 0.00903
index done: 14 RLRT_center: 0.00225 0.00982 RLRT_data: 0.00193 0.00889
index done: 15 RLRT_center: 0.00213 0.00948 RLRT_data: 0.00188 0.00879
index done: 16 RLRT_center: 0.00202 0.00932 RLRT_data: 0.00186 0.00882
index done: 17 RLRT_center: 0.00193 0.00958 RLRT_data: 0.00188 0.00935
index done: 18 RLRT_center: 0.00195 0.00994 RLRT_data: 0.00194 0.00987
index done: 19 RLRT_center: 0.00199 0.0104 RLRT_data: 0.00201 0.0105
index done: 20 RLRT_center: 0.00207 0.0109 RLRT_data: 0.00213 0.0113
index done: 21 RLRT_center: 0.00214 0.0113 RLRT_data: 0.00225 0.0121
index done: 22 RLRT_center: 0.00224 0.0117 RLRT_data: 0.00241 0.0129
index done: 53 RLRT_center: 0.00199 0.0104 RLRT_data: 0.00149 0.00793
index done: 54 RLRT_center: 0.00204 0.0108 RLRT_data: 0.00159 0.0084
index done: 55 RLRT_center: 0.00214 0.0113 RLRT_data: 0.0017 0.00893
index done: 56 RLRT_center: 0.00224 0.0117 RLRT_data: 0.00185 0.00949
index done: 57 RLRT_c

index done: 840 RLRT_center: 0.00235 0.0121 RLRT_data: 0.00182 0.00916
index done: 841 RLRT_center: 0.00246 0.0124 RLRT_data: 0.00194 0.00945
index done: 842 RLRT_center: 0.00246 0.0124 RLRT_data: 0.00194 0.00945
index done: 843 RLRT_center: 0.00246 0.0124 RLRT_data: 0.00194 0.00945
index done: 844 RLRT_center: 0.00246 0.0124 RLRT_data: 0.00194 0.00945
index done: 845 RLRT_center: 0.00259 0.0125 RLRT_data: 0.00206 0.00969
index done: 846 RLRT_center: 0.00272 0.0126 RLRT_data: 0.0022 0.00988
index done: 847 RLRT_center: 0.00286 0.0127 RLRT_data: 0.00234 0.01
index done: 848 RLRT_center: 0.00299 0.0126 RLRT_data: 0.00249 0.0102
index done: 849 RLRT_center: 0.00316 0.0126 RLRT_data: 0.00265 0.0102
index done: 850 RLRT_center: 0.00334 0.0125 RLRT_data: 0.00283 0.0103
index done: 851 RLRT_center: 0.00355 0.0124 RLRT_data: 0.00302 0.0104
index done: 852 RLRT_center: 0.00377 0.0123 RLRT_data: 0.00324 0.0104
index done: 853 RLRT_center: 0.00403 0.0123 RLRT_data: 0.00348 0.0105
index done: 854 

index done: 171 RLRT_center: 0.0011 0.00617 RLRT_data: 0.000791 0.00463
index done: 172 RLRT_center: 0.00115 0.00633 RLRT_data: 0.000846 0.0048
index done: 173 RLRT_center: 0.00121 0.00649 RLRT_data: 0.000909 0.00499
index done: 174 RLRT_center: 0.00128 0.00666 RLRT_data: 0.000979 0.00517
index done: 175 RLRT_center: 0.00135 0.00682 RLRT_data: 0.00106 0.00537
index done: 176 RLRT_center: 0.00144 0.00696 RLRT_data: 0.00115 0.00556
index done: 177 RLRT_center: 0.00153 0.0071 RLRT_data: 0.00124 0.00576
index done: 178 RLRT_center: 0.00163 0.00721 RLRT_data: 0.00135 0.00596
index done: 179 RLRT_center: 0.00174 0.00732 RLRT_data: 0.00147 0.00615
index done: 180 RLRT_center: 0.00186 0.00741 RLRT_data: 0.00159 0.00634
index done: 181 RLRT_center: 0.00198 0.00749 RLRT_data: 0.00173 0.00653
index done: 182 RLRT_center: 0.00212 0.00757 RLRT_data: 0.00188 0.00672
index done: 183 RLRT_center: 0.00226 0.00765 RLRT_data: 0.00205 0.0069
index done: 184 RLRT_center: 0.00242 0.00772 RLRT_data: 0.00222 

index done: 601 RLRT_center: 0.0161 0.0147 RLRT_data: 0.0188 0.0162
index done: 602 RLRT_center: 0.0166 0.0148 RLRT_data: 0.0195 0.0165
index done: 603 RLRT_center: 0.0175 0.0151 RLRT_data: 0.0211 0.017
index done: 604 RLRT_center: 0.0179 0.0153 RLRT_data: 0.0219 0.0173
index done: 605 RLRT_center: 0.0184 0.0154 RLRT_data: 0.0228 0.0176
index done: 606 RLRT_center: 0.0188 0.0155 RLRT_data: 0.0237 0.0178
index done: 694 RLRT_center: 0.0202 0.016 RLRT_data: 0.0145 0.0129
index done: 695 RLRT_center: 0.0207 0.0161 RLRT_data: 0.015 0.0132
index done: 696 RLRT_center: 0.0212 0.0162 RLRT_data: 0.0156 0.0135
index done: 697 RLRT_center: 0.0217 0.0164 RLRT_data: 0.0162 0.0137
index done: 698 RLRT_center: 0.0222 0.0165 RLRT_data: 0.0168 0.014
index done: 806 RLRT_center: 1.28e-05 0.000205 RLRT_data: 1e-05 0.000159
index done: 807 RLRT_center: 2.36e-05 0.000318 RLRT_data: 1.89e-05 0.000265
index done: 808 RLRT_center: 4.28e-05 0.000484 RLRT_data: 3.57e-05 0.000427
index done: 809 RLRT_center: 7.

...

index done: 938 RLRT_center: 0.000256 0.00183 RLRT_data: 0.000187 0.00142
index done: 939 RLRT_center: 0.000256 0.00183 RLRT_data: 0.000187 0.00142
index done: 940 RLRT_center: 0.000256 0.00183 RLRT_data: 0.000187 0.00142
index done: 941 RLRT_center: 0.000256 0.00183 RLRT_data: 0.000187 0.00142
index done: 942 RLRT_center: 0.000284 0.002 RLRT_data: 0.000209 0.00156
index done: 943 RLRT_center: 0.000284 0.002 RLRT_data: 0.000209 0.00156
index done: 944 RLRT_center: 0.000284 0.002 RLRT_data: 0.000209 0.00156
index done: 945 RLRT_center: 0.000284 0.002 RLRT_data: 0.000209 0.00156
index done: 946 RLRT_center: 0.000309 0.00215 RLRT_data: 0.000229 0.00169
index done: 947 RLRT_center: 0.000332 0.00229 RLRT_data: 0.000248 0.00181
index done: 948 RLRT_center: 0.000351 0.00241 RLRT_data: 0.000267 0.00192
index done: 949 RLRT_center: 0.000367 0.00251 RLRT_data: 0.000283 0.00203
index done: 950 RLRT_center: 0.000381 0.0026 RLRT_data: 0.000299 0.00213
index done: 951 RLRT_center: 0.000393 0.00268 R

index done: 1176 RLRT_center: 0.0139 0.0122 RLRT_data: 0.0149 0.0128
index done: 1177 RLRT_center: 0.0142 0.0123 RLRT_data: 0.0154 0.013
index done: 1178 RLRT_center: 0.0144 0.0124 RLRT_data: 0.0158 0.0132
index done: 1179 RLRT_center: 0.0147 0.0126 RLRT_data: 0.0163 0.0134
index done: 240 RLRT_center: 6.15e-05 0.000564 RLRT_data: 4.67e-05 0.000441
index done: 241 RLRT_center: 7.22e-05 0.000643 RLRT_data: 5.48e-05 0.000503
index done: 242 RLRT_center: 8.3e-05 0.000721 RLRT_data: 6.34e-05 0.000568
index done: 243 RLRT_center: 9.38e-05 0.000798 RLRT_data: 7.25e-05 0.000635
index done: 244 RLRT_center: 0.000104 0.000871 RLRT_data: 8.2e-05 0.000704
index done: 245 RLRT_center: 0.000115 0.000942 RLRT_data: 9.2e-05 0.000774
index done: 246 RLRT_center: 0.000135 0.00108 RLRT_data: 0.000113 0.000918
index done: 247 RLRT_center: 0.000145 0.00114 RLRT_data: 0.000125 0.000992
index done: 248 RLRT_center: 0.000154 0.0012 RLRT_data: 0.000136 0.00107
index done: 249 RLRT_center: 0.000164 0.00125 RLR

index done: 787 RLRT_center: 0.00481 0.00646 RLRT_data: 0.00438 0.00602
index done: 788 RLRT_center: 0.00495 0.00656 RLRT_data: 0.00459 0.0062
index done: 789 RLRT_center: 0.00509 0.00666 RLRT_data: 0.00481 0.00638
index done: 790 RLRT_center: 0.00524 0.00676 RLRT_data: 0.00503 0.00656
index done: 791 RLRT_center: 0.00538 0.00686 RLRT_data: 0.00527 0.00674
index done: 792 RLRT_center: 0.00553 0.00695 RLRT_data: 0.00551 0.00693
index done: 793 RLRT_center: 0.00568 0.00705 RLRT_data: 0.00576 0.00712
index done: 794 RLRT_center: 0.00582 0.00715 RLRT_data: 0.00601 0.00732
index done: 795 RLRT_center: 0.00612 0.00735 RLRT_data: 0.00655 0.00771
index done: 796 RLRT_center: 0.00627 0.00745 RLRT_data: 0.00684 0.00791
index done: 797 RLRT_center: 0.00642 0.00755 RLRT_data: 0.00713 0.00812
index done: 798 RLRT_center: 0.00657 0.00764 RLRT_data: 0.00743 0.00833
index done: 799 RLRT_center: 0.00672 0.00774 RLRT_data: 0.00774 0.00854
index done: 800 RLRT_center: 0.00687 0.00784 RLRT_data: 0.00806 0

index done: 1296 RLRT_center: 0.00809 0.00863 RLRT_data: 0.00889 0.00919
index done: 1297 RLRT_center: 0.00824 0.00873 RLRT_data: 0.00917 0.00936
index done: 1298 RLRT_center: 0.0084 0.00883 RLRT_data: 0.00946 0.00954
index done: 1299 RLRT_center: 0.00855 0.00892 RLRT_data: 0.00976 0.00972
index done: 1300 RLRT_center: 0.00871 0.00902 RLRT_data: 0.0101 0.00991
index done: 1301 RLRT_center: 0.00887 0.00912 RLRT_data: 0.0104 0.0101
index done: 229 RLRT_center: 2.84e-06 5.17e-05 RLRT_data: 2.75e-06 5.22e-05
index done: 230 RLRT_center: 3.91e-06 6.39e-05 RLRT_data: 3.8e-06 6.48e-05
index done: 231 RLRT_center: 5.3e-06 7.89e-05 RLRT_data: 5.21e-06 8.08e-05
index done: 232 RLRT_center: 7.07e-06 9.7e-05 RLRT_data: 7.07e-06 0.000101
index done: 233 RLRT_center: 9.27e-06 0.000119 RLRT_data: 9.49e-06 0.000125
index done: 234 RLRT_center: 1.19e-05 0.000144 RLRT_data: 1.25e-05 0.000154
index done: 235 RLRT_center: 1.51e-05 0.000172 RLRT_data: 1.63e-05 0.000189
index done: 236 RLRT_center: 1.86e-05

index done: 1067 RLRT_center: 4.12e-05 0.000383 RLRT_data: 5.08e-05 0.000469
index done: 1068 RLRT_center: 4.61e-05 0.000419 RLRT_data: 5.79e-05 0.000522
index done: 1069 RLRT_center: 5.12e-05 0.000455 RLRT_data: 6.52e-05 0.000575
index done: 1070 RLRT_center: 5.64e-05 0.00049 RLRT_data: 7.28e-05 0.000629
index done: 1071 RLRT_center: 6.17e-05 0.000525 RLRT_data: 8.06e-05 0.000684
index done: 1072 RLRT_center: 6.73e-05 0.000559 RLRT_data: 8.87e-05 0.000738
index done: 1223 RLRT_center: 0.000334 0.00133 RLRT_data: 0.000278 0.00107
index done: 1224 RLRT_center: 0.00036 0.00138 RLRT_data: 0.000301 0.00113
index done: 1225 RLRT_center: 0.000388 0.00143 RLRT_data: 0.000327 0.00118
index done: 1226 RLRT_center: 0.000417 0.00149 RLRT_data: 0.000354 0.00124
index done: 1227 RLRT_center: 0.000448 0.00154 RLRT_data: 0.000384 0.00131
index done: 1228 RLRT_center: 0.00048 0.0016 RLRT_data: 0.000415 0.00137
index done: 1229 RLRT_center: 0.000514 0.00166 RLRT_data: 0.000448 0.00144
index done: 1230 

In [15]:
# calculate all bin centering correction factors Ex, nu, W2 in qv or q2 bins
# qv*qv = Q2 + nu*nu
# Q2 = qv*qv - nu*nu

# bin centering correction for qv bin in Ex:
df["bc_qv_ex"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
                df.loc[index, 'bc_qv_ex']= (row['epsilon']*row['RL_fortran_qvcenter_ex']+0.5*((qvcenter**2
                                                        )/(qvcenter**2-row['nu']**2)
                                                    )*row['RT_fortran_qvcenter_ex']
                    )/(row['epsilon']*row['RL_fortran_qvdata_ex']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_ex'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index,'qvcenter:',qvcenter,'nu:',row['nu'],
                      'Ex:',row['Ex'],'RT_fortran_qvdata_ex:',row['RT_fortran_qvdata_ex'])
            

# bin centering correction for qv bin in nu:
df["bc_qv_nu"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
               df.loc[index, 'bc_qv_nu']= (row['epsilon']*row['RL_fortran_qvcenter_nu']+0.5*((qvcenter**2
                                                        )/(qvcenter**2-row['nu']**2)
                                                    )*row['RT_fortran_qvcenter_nu']
                    )/(row['epsilon']*row['RL_fortran_qvdata_nu']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_nu'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index,'qvcenter:',qvcenter,'nu:',row['nu'],
                      'Ex:',row['Ex'],'RT_fortran_qvdata_nu:',row['RT_fortran_qvdata_nu'])
                

# bin centering correction for qv bin in W2:
df["bc_qv_w2"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
               df.loc[index, 'bc_qv_w2']= (row['epsilon']*row['RL_fortran_qvcenter_w2']+0.5*((qvcenter**2
                                                        )/(
                                                            2*mass_nucleon*np.sqrt(qvcenter**2+row['W2'])-row['W2']-mass_nucleon**2
                                                        )
                                                    )*row['RT_fortran_qvcenter_w2']
                    )/(row['epsilon']*row['RL_fortran_qvdata_w2']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_w2'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index)


# bin centering correction for q2 bin in Ex:
df["bc_q2_ex"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        df.loc[index, 'bc_q2_ex']= (row['epsilon']*row['RL_fortran_q2center_ex']+0.5*((Q2center+row["nu"]**2
                                                    )/(Q2center)
                                                )*row['RT_fortran_q2center_ex']
                )/(row['epsilon']*row['RL_fortran_q2data_ex']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_ex'])
        
# bin centering correction for q2 bin in nu:
df["bc_q2_nu"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        df.loc[index, 'bc_q2_nu']= (row['epsilon']*row['RL_fortran_q2center_nu']+0.5*((Q2center+row["nu"]**2
                                                    )/(Q2center)
                                                )*row['RT_fortran_q2center_nu']
                )/(row['epsilon']*row['RL_fortran_q2data_nu']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_nu'])

# bin centering correction for q2 bin in W2:
df["bc_q2_w2"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        nu = (row['W2']+Q2center-mass_nucleon**2)/(2*mass_nucleon)
        qv2center = Q2center+nu**2
        df.loc[index, 'bc_q2_w2']= (row['epsilon']*row['RL_fortran_q2center_w2']+0.5*(
                                                        qv2center/(Q2center)
                                                )*row['RT_fortran_q2center_w2']
                )/(row['epsilon']*row['RL_fortran_q2data_w2']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_w2'])


ZeroDivisionError, index: 102 qvcenter: 0.991 nu: 1.0251 Ex: 0.9957626635873631 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 103 qvcenter: 0.991 nu: 1.0436 Ex: 1.0142626635873633 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 104 qvcenter: 0.991 nu: 1.0622 Ex: 1.0328626635873632 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 105 qvcenter: 0.991 nu: 1.0807 Ex: 1.0513626635873632 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 106 qvcenter: 0.991 nu: 1.0992 Ex: 1.0698626635873631 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 107 qvcenter: 0.991 nu: 1.1177 Ex: 1.088362663587363 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 108 qvcenter: 0.991 nu: 1.1362 Ex: 1.1068626635873633 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 109 qvcenter: 0.991 nu: 1.1548 Ex: 1.1254626635873632 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 461 qvcenter: 0.991 nu: 1.033 Ex: 1.0009557891840517 RT_fortran_qvdata_ex: 0.0
ZeroDivisionError, index: 462 qvcenter: 0.991 nu

In [16]:
# df.to_csv("df_fortran_Feb19.csv",index=False)
# df.to_csv("df_fortran_Feb20.csv",index=False)
# df.to_csv('df_fortran_Goldemberg_Feb23.csv',index=False)
# df.to_csv('df_fortran_Feb24.csv',index=False)
# df.to_csv('df_fortran_March1.csv',index=False)
# df.to_csv('DataSet13_March15.csv',index=False)
# df.to_csv('Barreau/Barreau_cross_April12.csv',index=False)
# df.to_csv('Sheren_April12_fortran.csv',index=False)
df.to_csv('Sheren/Sheren_May1.csv',index=False)







In [17]:
df

,Z,A,E0,ThetaDeg,nu,cross,error,dataSet,ThetaRad,sin2(T/2),...,RL_fortran_q2center_w2,RT_fortran_q2center_w2,RL_fortran_q2data_w2,RT_fortran_q2data_w2,bc_qv_ex,bc_qv_nu,bc_qv_w2,bc_q2_ex,bc_q2_nu,bc_q2_w2
0,6,12,1.2044,45.0,0.1702,41.1,1.59,12,0.785398,0.146447,...,0.000041,0.000469,0.000036,0.000428,6.443982,7.963276,0.529198,0.561687,0.525555,1.113622
1,6,12,1.2044,45.0,0.1857,65.8,2.02,12,0.785398,0.146447,...,0.000073,0.000703,0.000065,0.000659,6.081272,7.217185,0.654282,0.501241,0.464999,1.087953
2,6,12,1.2044,45.0,0.2011,106.0,2.49,12,0.785398,0.146447,...,0.000124,0.001030,0.000115,0.001000,5.552772,6.266370,0.763178,0.447317,0.411936,1.053548
3,6,12,1.2044,45.0,0.2166,160.0,3.18,12,0.785398,0.146447,...,0.000200,0.001480,0.000199,0.001500,4.838996,5.214716,0.864939,0.401460,0.367372,1.008107
4,6,12,1.2044,45.0,0.2321,233.0,4.02,12,0.785398,0.146447,...,0.000303,0.002030,0.000327,0.002190,4.120560,4.247677,0.964153,0.363645,0.333795,0.946617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,6,12,4.6286,25.0,2.7422,168.0,3.30,12,0.436332,0.046846,...,0.008240,0.008730,0.009170,0.009360,0.019104,0.007918,0.572606,0.855131,0.856358,0.912420
1298,6,12,4.6286,25.0,2.7567,174.0,3.38,12,0.436332,0.046846,...,0.008400,0.008830,0.009460,0.009540,0.021475,0.009287,0.578975,0.840808,0.841730,0.902818
1299,6,12,4.6286,25.0,2.7713,176.0,3.44,12,0.436332,0.046846,...,0.008550,0.008920,0.009760,0.009720,0.024157,0.010851,0.584971,0.826242,0.826446,0.892312
1300,6,12,4.6286,25.0,2.7859,178.0,3.56,12,0.436332,0.046846,...,0.008710,0.009020,0.010100,0.009910,0.027208,0.012628,0.590005,0.811840,0.811392,0.881750


In [ ]:
# calculate all bin centering correction factors: in Ex or in nu? for qv or for q2?
# qv*qv = Q2 + nu*nu
# Q2 = qv*qv - nu*nu

# bin centering correction for qv bin in Ex:
df["bc_qv_ex"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
                df.loc[index,'bc_qv_ex']= (row['epsilon']*row['RL_fortran_qvcenter_ex']+0.5*((row['qvcenter']**2
                                                        )/(row['qvcenter']**2-row['nu']**2)
                                                    )*row['RT_fortran_qvcenter_ex']
                    )/(row['epsilon']*row['RL_fortran_qvdata_ex']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_ex'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index,'qvcenter:',qvcenter,'nu:',row['nu'],
                      'Ex:',row['Ex'],'RT_fortran_qvdata_ex:',row['RT_fortran_qvdata_ex'])
            

# bin centering correction for qv bin in nu:
df["bc_qv_nu"]=1.0
# for qvcenter in qvcenters:
#     picked = df.loc[df["qvcenter"]==qvcenter]

#     for index, row in picked.iterrows():
#         if row['qvcenter']!= float('NaN'):
#             if row['Ex']<=0.025:
#                 continue
#             try:
#                row['bc_qv_nu']= (row['epsilon']*row['RL_fortran_qvcenter_nu']+0.5*((row['qvcenter']**2
#                                                         )/(row['qvcenter']**2-row['nu']**2)
#                                                     )*row['RT_fortran_qvcenter_nu']
#                     )/(row['epsilon']*row['RL_fortran_qvdata_nu']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_nu'])
#             except ZeroDivisionError:
#                 print('ZeroDivisionError, index:',index,'qvcenter:',qvcenter,'nu:',row['nu'],
#                       'Ex:',row['Ex'],'RT_fortran_qvdata_nu:',row['RT_fortran_qvdata_nu'])
                

# bin centering correction for qv bin in W2:
df["bc_qv_w2"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]

    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN'):
            if row['Ex']<=0.025:
                continue
            try:
               df.loc[index,'bc_qv_w2']= (row['epsilon']*row['RL_fortran_qvcenter_w2']+0.5*((row['qvcenter']**2
                                                        )/(
                                                            2*mass_nucleon*np.sqrt(row['qvcenter']**2+row['W2'])-row['W2']-mass_nucleon**2
                                                        )
                                                    )*row['RT_fortran_qvcenter_w2']
                    )/(row['epsilon']*row['RL_fortran_qvdata_w2']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata_w2'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index)


# bin centering correction for q2 bin in Ex:
df["bc_q2_ex"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        df.loc[index,'bc_q2_ex']= (row['epsilon']*row['RL_fortran_q2center_ex']+0.5*((Q2center+row["nu"]**2
                                                    )/(row["Q2center"])
                                                )*row['RT_fortran_q2center_ex']
                )/(row['epsilon']*row['RL_fortran_q2data_ex']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_ex'])
        
# bin centering correction for q2 bin in nu:
# df["bc_q2_nu"]=1.0
# for Q2center in Q2centers:
#     picked = df.loc[df["Q2center"]==Q2center]

#     for index, row in picked.iterrows():
#         if row['Ex']<=0.025:
#             continue
#         row['bc_q2_nu']= (row['epsilon']*row['RL_fortran_q2center_nu']+0.5*((Q2center+row["nu"]**2
#                                                     )/(row["Q2center"])
#                                                 )*row['RT_fortran_q2center_nu']
#                 )/(row['epsilon']*row['RL_fortran_q2data_nu']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_nu'])

# bin centering correction for q2 bin in W2:
df["bc_q2_w2"]=1.0
for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]

    for index, row in picked.iterrows():
        if row['Ex']<=0.025:
            continue
        nu = (row['W2']+Q2center-mass_nucleon**2)/(2*mass_nucleon)
        qv2center = Q2center+nu**2
        df.loc[index,'bc_q2_w2']= (row['epsilon']*row['RL_fortran_q2center_w2']+0.5*(
                                                        qv2center/(row["Q2center"])
                                                )*row['RT_fortran_q2center_w2']
                )/(row['epsilon']*row['RL_fortran_q2data_w2']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data_w2'])


In [ ]:
df['bc_qv_w2'].hist(bins=100)

In [ ]:
# read from circ jupyter, df_qv_analysis.csv, to do analysis in qv
df = pd.read_csv("df_qv_analysis.csv")
for i in range(0,len(qvcenters)):
    qvcenter = qvcenters[i]
    plt.figure(figsize=(5,4))
    picked = df.loc[df["qvcenter"]==qvcenter]
    plt.scatter(picked['nu'],picked['RL_fortran_qvdata'],label='data',s=2)
    plt.scatter(picked['nu'],picked['RL_fortran_qvcenter'],label='center',s=2)
    plt.title('qvcenter = '+str(qvcenter))
    plt.xlabel('nu')
    plt.ylabel('RL')
    plt.legend()
    plt.xlim(0,None)
    plt.ylim(0,None)
for i in range(0,len(qvcenters)):
    qvcenter = qvcenters[i]
    plt.figure(figsize=(5,4))
    picked = df.loc[df["qvcenter"]==qvcenter]
    plt.scatter(picked['nu'],picked['RT_fortran_qvdata'],label='data',s=2)
    plt.scatter(picked['nu'],picked['RT_fortran_qvcenter'],label='center',s=2)
    plt.title('qvcenter = '+str(qvcenter))
    plt.xlabel('nu')
    plt.ylabel('RT')
    plt.legend()
    plt.xlim(0,None)
    plt.ylim(0,None)

In [ ]:
# calculate capital sigma, qv bin-centering correction using exact fortran RL, RT
# qv*qv = Q2 + nu*nu
# Q2 = qv*qv - nu*nu
df["bin_centering_correction"]=1.0
for index, row in df.iterrows():
    if row['Ex']<=0.025:
        continue

    df.loc[index,'bin_centering_correction']= (row['epsilon']*row['RL_fortran_qvcenter']+0.5*((row['qvcenter']**2
                                                )/(row['qvcenter']**2-row['nu']**2)
                                               )*row['RT_fortran_qvcenter']
            )/(row['epsilon']*row['RL_fortran_qvdata']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_qvdata'])
df["Hbc_Sig(GeV)"]=df["bin_centering_correction"]*df["Hcc_Sig(GeV)"]
df["Hbc_error(GeV)"]=df["bin_centering_correction"]*df["Hcc_error(GeV)"]

In [ ]:
# Jan 12: read Qvedges, Qvcenter_datas from saved csv files
Qvedges = []
for qvbin in qvbins:
    Qvedge = pd.read_csv('Qvedges/'+f'Qvedge_{qvbin}.csv',index_col = False)
    Qvedge.columns = ['qv','q2','ex','nu','RT','RL','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
    Qvedge['W2']=mass_nucleon**2+2*mass_nucleon*Qvedge['nu']-Qvedge['q2']
    Qvedges.append(Qvedge)
Qvcenter_datas = []
for qvcenter in qvcenters:
    Qvcenter_data = pd.read_csv('Qvedges/'+f'Qvedge_{qvcenter}.csv',index_col = False)
    Qvcenter_data.columns = ['qv','q2','ex','nu','RT','RL','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
    Qvcenter_data['W2']= mass_nucleon**2+2*mass_nucleon*Qvcenter_data['nu']-Qvcenter_data['q2']
    Qvcenter_datas.append(Qvcenter_data)

In [ ]:
# Jan 11: reproduce the graphs, use qvbin, plot in nu

def linear_model(x, a, b):
    return a * x + b

Yamaguchi_qvs = [0.148,0.167,0.205,0.24,0.3]

for i in range(0,len(qvcenters)):

    
    
    qvcenter = qvcenters[i]
    qvcenter_data = Qvcenter_datas[i]
    qv2center = qvcenter**2
    qv_bot = qvbins[i]
    qv_top = qvbins[i+1]
    qvbin_name = qvbin_names[i]
    W2width = W2widths[i]
    bin_data = df.loc[(df['q3momt'] >= qv_bot) & (df['q3momt'] <= qv_top)]

    nu_grid = np.arange(0.0,0.5,0.01)
    RLs = []
    RLerrs = []
    RTs = []
    RTerrs = []
    nus = []
    RxQ2s = []
    RxQ2_errs = []
    for nu in nu_grid:
        # q3momt_squared = Q2center+nu**2
        Q2 = qv2center-nu**2
        W2 = mass_nucleon**2+2*mass_nucleon*nu-Q2
        picked = bin_data.loc[(W2<=df["W2"]) & (df["W2"]<=(W2+2*W2width))]
        # picked = picked.drop_duplicates(keep="last")
        x = np.array(picked["epsilon"].values)
        y = np.array(picked["Hbc_Sig(GeV)"].values)
        y_err = np.array(picked["Hbc_error(GeV)"].values)
        
        if len(y)>=2:
            if len(y)==2:
                # print("len(y)==2")
                # duplicated_values = np.concatenate([original_values + 0.1 * original_errors,
                #                     original_values - 0.1 * original_errors])
                # duplicated_errors = original_errors * 1.414
                # duplicated_errors = np.repeat(duplicated_errors, 2)  # Repeat each error twice
                x = np.concatenate([x, x])
                y = np.concatenate([y+0.1*y_err, y-0.1*y_err])
                y_err = y_err * 1.414
                y_err = np.concatenate([y_err, y_err])
            params, covariance = curve_fit(linear_model, x, y, sigma=y_err)
            a_opt, b_opt = params
            a_err, b_err = np.sqrt(np.diag(covariance))
            RL = a_opt/1000
            RT = (2*b_opt*Q2/qv2center)/1000
            if RL>=0 and RT>=0:
                RLs.append(RL)
                RLerr = a_err/1000 
                RLerrs.append(RLerr)
                RTs.append(RT)
                RTerr = (2*b_err*Q2/qv2center)/1000
                RTerrs.append(RTerr)
                RxQ2s.append( (2*Q2/qv2center)*RL/RT)
                RxQ2_err = np.abs ((2*Q2/qv2center)) * (RL/RT)* np.sqrt( (RLerr/RL)**2 + (RTerr/RT)**2 )
                RxQ2_errs.append(RxQ2_err)
                # W2s.append(W2)
                nus.append(nu)


    # fig, axs = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns

    tick_spacing = 0.05
    try:
        responseq2_plotting = qvcenter_data.loc[qvcenter_data['nu']<=np.max(nus)]
    except ValueError:
        print("ValueError: no data points")
    axs[0].plot(responseq2_plotting['nu'],responseq2_plotting["RL"],label="Fortran fit", linestyle='dotted',color='black')    
    axs[0].errorbar(nus, RLs, yerr = RLerrs, color='orange', fmt ='.',capsize=3,markersize='3')
    axs[0].scatter(nus, RLs,marker='D',s=6,color='red',label="$Q3_{center}:$"+str(qvcenter))
    axs[0].set_title("$Q3$:"+qvbin_name)
    axs[0].set_xlabel("$\\nu(GeV)$")
    axs[0].set_ylabel("$R_L$")
    axs[0].xaxis.set_minor_locator(AutoMinorLocator())
    axs[0].yaxis.set_minor_locator(AutoMinorLocator())
    axs[0].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    axs[0].tick_params(axis='both', direction='in',which='minor',top=True,right=True)


    axs[1].plot(responseq2_plotting['nu'],responseq2_plotting["RT"],label="Fortran fit", linestyle='dotted',color='black')
    axs[1].errorbar(nus, RTs, yerr = RTerrs, color='orange', fmt ='.',capsize=3,markersize='3')
    axs[1].scatter(nus, RTs,marker='D',s=6,color='red',label="$Q3_{center}:$"+str(qvcenter))
    axs[1].set_title("$Q3$:"+qvbin_name)
    axs[1].set_xlabel("$\\nu(GeV)$")
    axs[1].set_ylabel("$R_T$")
    axs[1].xaxis.set_minor_locator(AutoMinorLocator())
    axs[1].yaxis.set_minor_locator(AutoMinorLocator())
    axs[1].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    axs[1].tick_params(axis='both', direction='in',which='minor',top=True,right=True)

    if qvcenter in Ryan['qvcenter'].unique():
        Ryan_data = Ryan.loc[Ryan['qvcenter']==qvcenter]
        axs[1].errorbar(Ryan_data['nu'],Ryan_data["RT"], yerr = Ryan_data["RTerr"], color='lightgreen', fmt ='.',capsize=3,markersize='3')
        axs[1].scatter(Ryan_data['nu'],Ryan_data["RT"],marker='D',s=6,color='green',label="Ryan")
        axs[1].errorbar(Ryan_data['nu'],Ryan_data["RT_bc"], yerr = Ryan_data["RTerr_bc"], color='plum', fmt ='.',capsize=3,markersize='3')
        axs[1].scatter(Ryan_data['nu'],Ryan_data["RT_bc"],marker='D',s=6,color='purple',label="Ryan bin_centered")
    
    if qvcenter in Yamaguchi_qvs:
        Yam_data = pd.read_excel('Yamaguchi_plotting.xlsx',sheet_name='qv_'+str(qvcenter))
        # Yam_data['nu_RL'] = - mass_C12 + np.sqrt(mass_C12**2 + qvcenter**2 + 2*mass_C12*Yam_data['EX_RL'])
        # Yam_data['nu_RT'] = - mass_C12 + np.sqrt(mass_C12**2 + qvcenter**2 + 2*mass_C12*Yam_data['EX_RT'])
        # Ex = nu-(qv^2-nu^2)/(2M)
        # nu = - M + sqrt(M^2 + qv^2 + 2M*Ex)
        # solve quadratic equation
        
        # axs[0].errorbar(Yam_data['nu_RL'],Yam_data["RL"], yerr = Yam_data["RL"]*0.03, color='lightpink', fmt ='.',capsize=3,markersize='3')
        axs[0].scatter(Yam_data['nu_RL'],Yam_data["RL"],marker='D',s=6,color='skyblue',label="Yamaguchi")
        # axs[1].errorbar(Yam_data['nu_RT'],Yam_data["RT"], yerr = Yam_data["RT"]*0.03, color='lightpink', fmt ='.',capsize=3,markersize='3')
        axs[1].scatter(Yam_data['nu_RT'],Yam_data["RT"],marker='D',s=6,color='skyblue',label="Yamaguchi")        


    axs[0].legend()
    axs[0].set_xlim(0.0,None)
    axs[0].set_ylim(0.0,None)

    axs[1].legend()
    axs[1].set_xlim(0.0,None)
    axs[1].set_ylim(0.0,None)


    plt.tight_layout()  # Optional, for better spacing between subplots
    plt.show()

    fig.savefig('figs/FitNu_Qv_'+str(qvcenter)+'.png')


In [ ]:
# read from circ jupyter, df_Q2_analysis.csv, to do analysis in Q2
df = pd.read_csv("df_Q2_analysis.csv")
for i in range(0,len(Q2centers)):
    Q2center = Q2centers[i]
    plt.figure(figsize=(5,4))
    picked = df.loc[df["Q2center"]==Q2center]
    plt.scatter(picked['nu'],picked['RL_fortran_q2data'],label='data',s=2)
    plt.scatter(picked['nu'],picked['RL_fortran_q2center'],label='center',s=2)
    plt.title('Q2center = '+str(Q2center))
    plt.xlabel('nu')
    plt.ylabel('RL')
    plt.legend()
    plt.xlim(0,None)
    plt.ylim(0,None)
for i in range(0,len(Q2centers)):
    Q2center = Q2centers[i]
    plt.figure(figsize=(5,4))
    picked = df.loc[df["Q2center"]==Q2center]
    plt.scatter(picked['nu'],picked['RT_fortran_q2data'],label='data',s=2)
    plt.scatter(picked['nu'],picked['RT_fortran_q2center'],label='center',s=2)
    plt.title('Q2center = '+str(Q2center))
    plt.xlabel('nu')
    plt.ylabel('RT')
    plt.legend()
    plt.xlim(0,None)
    plt.ylim(0,None)


In [ ]:
# calculate capital sigma, Q2 bin-centering correction using exact fortran RL, RT
df['bin_centering_correction'] = 1.0
for index, row in df.iterrows():
    if row['Ex']<=0.025:
        continue
    df.loc[index,'bin_centering_correction']= (row['epsilon']*row['RL_fortran_q2center']+0.5*((row['Q2center']+row['nu']**2
                                                )/(row['Q2center'])
                                               )*row['RT_fortran_q2center']
            )/(row['epsilon']*row['RL_fortran_q2data']+0.5*(row['q3momt_squared']/row['Q2eff'])*row['RT_fortran_q2data'])
df["Hbc_Sig(GeV)"]=df["bin_centering_correction"]*df["Hcc_Sig(GeV)"]
df["Hbc_error(GeV)"]=df["bin_centering_correction"]*df["Hcc_error(GeV)"]

In [ ]:
# Jan 20: read Q2edges, Q2center_datas from saved csv files
Q2edges = []
for bin in bins:
    Q2edge = pd.read_csv('Q2edges/'+f'Q2edge_{bin}.csv',index_col = False)
    Q2edge.columns = ['qv','q2','ex','nu','RT','RL','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
    Q2edge['W2']=mass_nucleon**2+2*mass_nucleon*Q2edge['nu']-Q2edge['q2']
    Q2edges.append(Q2edge)

Q2center_datas = []
for Q2center in Q2centers:
    Q2center_data = pd.read_csv('Q2edges/'+f'Q2edge_{Q2center}.csv',index_col = False)
    Q2center_data.columns = ['qv','q2','ex','nu','RT','RL','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
    Q2center_data['W2']= mass_nucleon**2+2*mass_nucleon*Q2center_data['nu']-Q2center
    Q2center_datas.append(Q2center_data)

In [ ]:
# reproduce the graphs, using saved Q2center datas, plot in nu; coulomb & bin centering corrections applied


fortran_program = "Eric/responseq2-oct2/responseq2"

def linear_model(x, a, b):
    return a * x + b

for bin in Q2bins:
    # plt.figure(figsize=(15,5))
    Q2center = Q2bin_to_Q2center[bin]
    Q2center_index = Q2centers.index(Q2center)
    Q2center_data = Q2center_datas[Q2center_index]

    # W2_grid = np.arange(mass_nucleon**2+2*mass_nucleon*0.028-Q2center,
    #                      mass_nucleon**2+2*mass_nucleon*0.3-Q2center,
    #                      0.01)
    nu_grid = np.arange(0.0,1,0.01)
    
    W2width = Q2bin_to_W2width[bin]
    RLs = []
    RLerrs = []
    RTs = []
    RTerrs = []
    nus = []
    # W2s = []
    RxQ2s = []
    RxQ2_errs = []

    for nu in nu_grid:
    # for W2 in W2_grid:
        
        # nu = (W2 + Q2center - mass_nucleon**2)/(2*mass_nucleon)
        W2 = mass_nucleon**2+2*mass_nucleon*nu-Q2center
        q3momt_squared = Q2center+nu**2
        # focus on Ex > 0.035 GeV entries
        picked = df.loc[(W2<=df["W2"]) & (df["W2"]<=(W2+2*W2width)) & (df["bin"] == bin) ]
        # picked = df.loc[(W2<=df["W2"]) & (df["W2"]<=(W2+2*W2width)) & (df["bin"] == bin) & (df["Ex"]>0.035)]
        picked = picked.drop_duplicates(keep="last")
        x = np.array(picked["epsilon"].values)
        y = np.array(picked["Hbc_Sig(GeV)"].values)
        y_err = np.array(picked["Hbc_error(GeV)"].values)
        if len(y)>=2:
            if len(y)==2:
                # duplicate the data points 
                x = np.concatenate([x, x])
                y = np.concatenate([y+0.1*y_err, y-0.1*y_err])
                y_err = y_err * 1.414
                y_err = np.concatenate([y_err, y_err])
            params, covariance = curve_fit(linear_model, x, y, sigma=y_err)
            a_opt, b_opt = params
            a_err, b_err = np.sqrt(np.diag(covariance))
            # if (RL>=0 and RT>=0):
            RL = a_opt/1000
            RT = (2*b_opt*Q2center/q3momt_squared)/1000
            if RL>=0 and RT>=0:
                RLs.append(RL)
                RLerr = a_err/1000 
                RLerrs.append(RLerr)
                RTs.append(RT)
                RTerr = (2*b_err*Q2center/q3momt_squared)/1000
                RTerrs.append(RTerr)
                RxQ2s.append( (2*Q2center/q3momt_squared)*RL/RT)
                RxQ2_err = np.abs ((2*Q2center/q3momt_squared)) * (RL/RT)* np.sqrt( (RLerr/RL)**2 + (RTerr/RT)**2 )
                RxQ2_errs.append(RxQ2_err)
                # W2s.append(W2)
                nus.append(nu)

    # fig, axs = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns
    tick_spacing = 0.05/5

   
    try:
        responseq2_plotting = Q2center_data.loc[Q2center_data['nu']<=np.max(nus)]
    except ValueError:
        print("ValueError: no data points")
    axs[0].plot(responseq2_plotting['nu'],responseq2_plotting["RL"],label="Fortran fit $Q^2_{center}$", linestyle='dotted',color='black')    
    axs[0].errorbar(nus, RLs, yerr = RLerrs, color='orange', fmt ='.',capsize=3,markersize='3')
    axs[0].scatter(nus, RLs,marker='D',s=6,color='red',label="Bin centered fit")
    axs[0].set_title("$Q^2$:"+bin)
    axs[0].set_xlabel("$\\nu(GeV)$")
    axs[0].set_ylabel("$R_L$")
    axs[0].xaxis.set_minor_locator(AutoMinorLocator())
    axs[0].yaxis.set_minor_locator(AutoMinorLocator())
    axs[0].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    axs[0].tick_params(axis='both', direction='in',which='minor',top=True,right=True)
    axs[0].legend()
    axs[0].set_xlim(0,None)
    axs[0].set_ylim(0,None)

    
    # axs[1].plot(responseq2["W2"],responseq2["RTTOT"],label="responseq2",linestyle='dotted')

    # responseq2_W2 =  mass_nucleon**2+2*mass_nucleon*Q2center_data['nu']-Q2center
    # responseq2_W2_plotting = responseq2_W2.loc[responseq2_W2['nu']<=W2s.max()]

    axs[1].plot(responseq2_plotting['nu'],responseq2_plotting["RT"],label="Fortran fit $Q^2_{center}$", linestyle='dotted',color='black')
    axs[1].errorbar(nus, RTs, yerr = RTerrs, color='orange', fmt ='.',capsize=3,markersize='3')
    axs[1].scatter(nus, RTs,marker='D',s=6,color='red',label="Bin centered fit")
    axs[1].set_title("$Q^2$:"+bin)
    axs[1].set_xlabel("$\\nu(GeV)$")
    axs[1].set_ylabel("$R_T$")
    axs[1].xaxis.set_minor_locator(AutoMinorLocator())
    axs[1].yaxis.set_minor_locator(AutoMinorLocator())
    axs[1].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    axs[1].tick_params(axis='both', direction='in',which='minor',top=True,right=True)

    if Q2center == Q2centers[0]:
        axs[1].errorbar(Photon_plotting['nu'], Photon_plotting["RT"], yerr = Photon_plotting["error"], color='lightgreen', fmt ='.',capsize=1,markersize='3')
        axs[1].scatter(Photon_plotting['nu'], Photon_plotting["RT"],marker='D',s=2,color='green',label="Photon Prod")


    axs[1].legend()
    axs[1].set_xlim(0,None)
    axs[1].set_ylim(0,None)
    


    # axs[2].scatter(responseq2['W2'],responseq2['RxQ2'],label="responseq2",s=6,marker='D',color='lightblue')

    # axs[2].scatter(nus, RxQ2s,marker='D',s=6,color='red',label="$Q^2_{center}:$"+str(Q2bin_to_Q2center[bin]))
    # axs[2].errorbar(nus, RxQ2s, yerr = RxQ2_errs, color='orange', fmt ='.',capsize=3,markersize='3')
    # axs[2].set_title("$Q^2$:"+bin)
    # axs[2].set_xlabel("$\\nu(GeV)$")
    # axs[2].set_ylabel("$R(x,Q^2)$")
    # axs[2].xaxis.set_minor_locator(AutoMinorLocator())
    # axs[2].yaxis.set_minor_locator(AutoMinorLocator())
    # axs[2].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    # axs[2].tick_params(axis='both', direction='in',which='minor',top=True,right=True)
    # axs[2].legend()

    # do it again for Hcc
    RLs = []
    RLerrs = []
    RTs = []
    RTerrs = []
    nus = []
    # W2s = []
    RxQ2s = []
    RxQ2_errs = []

    for nu in nu_grid:
    # for W2 in W2_grid:
        
        # nu = (W2 + Q2center - mass_nucleon**2)/(2*mass_nucleon)
        W2 = mass_nucleon**2+2*mass_nucleon*nu-Q2center
        q3momt_squared = Q2center+nu**2
        # focus on Ex > 0.035 GeV entries
        picked = df.loc[(W2<=df["W2"]) & (df["W2"]<=(W2+2*W2width)) & (df["bin"] == bin) ]
        # picked = df.loc[(W2<=df["W2"]) & (df["W2"]<=(W2+2*W2width)) & (df["bin"] == bin) & (df["Ex"]>0.035)]
        picked = picked.drop_duplicates(keep="last")
        x = np.array(picked["epsilon"].values)
        y = np.array(picked["Hcc_Sig(GeV)"].values)
        y_err = np.array(picked["Hcc_error(GeV)"].values)
        if len(y)>=2:
            if len(y)==2:
                # duplicate the data points 
                x = np.concatenate([x, x])
                y = np.concatenate([y+0.1*y_err, y-0.1*y_err])
                y_err = y_err * 1.414
                y_err = np.concatenate([y_err, y_err])
            params, covariance = curve_fit(linear_model, x, y, sigma=y_err)
            a_opt, b_opt = params
            a_err, b_err = np.sqrt(np.diag(covariance))
            # if (RL>=0 and RT>=0):
            RL = a_opt/1000
            RT = (2*b_opt*Q2center/q3momt_squared)/1000
            if RL>=0 and RT>=0:
                RLs.append(RL)
                RLerr = a_err/1000 
                RLerrs.append(RLerr)
                RTs.append(RT)
                RTerr = (2*b_err*Q2center/q3momt_squared)/1000
                RTerrs.append(RTerr)
                RxQ2s.append( (2*Q2center/q3momt_squared)*RL/RT)
                RxQ2_err = np.abs ((2*Q2center/q3momt_squared)) * (RL/RT)* np.sqrt( (RLerr/RL)**2 + (RTerr/RT)**2 )
                RxQ2_errs.append(RxQ2_err)
                # W2s.append(W2)
                nus.append(nu)
    axs[0].scatter(nus, RLs,marker='D',s=6,color='blue',label="Non-bc fit")
    axs[0].errorbar(nus, RLs, yerr = RLerrs, color='skyblue', fmt ='.',capsize=3,markersize='3')
    axs[0].set_title("$Q^2$:"+bin+" $Q^2_{center}:$"+str(Q2center))
    axs[0].set_xlabel("$\\nu(GeV)$")
    axs[0].set_ylabel("$R_L$")
    axs[0].xaxis.set_minor_locator(AutoMinorLocator())
    axs[0].yaxis.set_minor_locator(AutoMinorLocator())
    axs[0].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    axs[0].tick_params(axis='both', direction='in',which='minor',top=True,right=True)
    axs[0].legend()
    axs[0].set_xlim(0,None)
    axs[0].set_ylim(0,None)

    
    # axs[1].plot(responseq2["W2"],responseq2["RTTOT"],label="responseq2",linestyle='dotted')

    # responseq2_W2 =  mass_nucleon**2+2*mass_nucleon*Q2center_data['nu']-Q2center
    # responseq2_W2_plotting = responseq2_W2.loc[responseq2_W2['nu']<=W2s.max()]

    axs[1].scatter(nus, RTs,marker='D',s=6,color='blue',label="Non-bc fit")
    axs[1].errorbar(nus, RTs, yerr = RTerrs, color='deepskyblue', fmt ='.',capsize=3,markersize='3')
    axs[1].set_title("$Q^2$:"+bin+" $Q^2_{center}:$"+str(Q2center))
    axs[1].set_xlabel("$\\nu(GeV)$")
    axs[1].set_ylabel("$R_T$")
    axs[1].xaxis.set_minor_locator(AutoMinorLocator())
    axs[1].yaxis.set_minor_locator(AutoMinorLocator())
    axs[1].tick_params(axis='both', direction='out',which='major',top=True,right=True)
    axs[1].tick_params(axis='both', direction='in',which='minor',top=True,right=True)
    axs[1].legend()
    axs[1].set_xlim(0,None)
    axs[1].set_ylim(0,None)
    




    plt.tight_layout()  # Optional, for better spacing between subplots
    plt.show()
    fig.savefig('figs/FitNu_Q2_'+str(Q2center)+'.png')


In [ ]:
# df.to_csv("df_fortran_Feb15.csv",index=False)
df.to_csv("df_fortran_Feb19.csv",index=False)

In [ ]:
df.to_csv("df_qv_analysis_Ex.csv",index=False)

In [ ]:
df.to_csv("df_q2_analysis_Ex.csv",index=False)

In [ ]:
# Jan 22: compute Fortran fit for the cross section


def remove_asterisks(input_string):
    return input_string.rstrip('*')

def remove_asterisks_list(string_list):
    return [remove_asterisks(s) for s in string_list]

df['sigtot','sigqe','sigie','sigmec','nuccstot','signonuc']=0.0
fortran_folder = "Eric/cscompred/"
for index, row in df.iterrows():
    # calculate center
    E0 = row['E0']
    theta = row['ThetaDeg']
    nu = row['nu']
    output = ''

    with subprocess.Popen(fortran_folder+'modplotred_ziggy', stdin=subprocess.PIPE, stdout=subprocess.PIPE, text=True, shell=True) as process:
        input_data = str(E0)+' '+str(theta)+' '+str(nu)
        output, _ = process.communicate(input_data)

    print('index:',index,'output:',output)
    output = output.split()

#     output = remove_asterisks_list(output)
#     print('index:',index,'output:',output)
#     output = np.array(list(map(float, output)))
    
#     if len(output)==0:
#         print('ERROR index:',index,'no result. E0,theta,nu:',E0,theta,nu)
#         continue
#     # output format: ep,theta,nu,w2,q2,sigtot,sigqe,sigie,sigmec,nuccstot,signonuc
#     df.loc[index,'sigtot']=output[5]
#     df.loc[index,'sigqe']=output[6]
#     df.loc[index,'sigie']=output[7]
#     df.loc[index,'sigmec']=output[8]
#     df.loc[index,'nuccstot']=output[9]
#     df.loc[index,'signonuc']=output[10]
#     print('index:',index,'E0,theta,nu:',E0,theta,nu,'done: sigtot =',output[5])
    

In [ ]:
fortran_program = "Eric/response/responseq2"
print("Q2edge for Q2centers computing...")
for Q2center in [1.250,1.750,2.250,2.750,3.250,3.750]:
    if os.path.exists('Q2edges/'+f'Q2edge_{Q2center}.csv'):
        print('Q2edge_'+str(bin)+'.csv already exists')
        continue
    user_input = str(Q2center)
    process = subprocess.Popen(fortran_program, stdin=subprocess.PIPE, text=True, bufsize=1, universal_newlines=True)
    process.stdin.write(user_input + '\n')
    process.stdin.flush()
    process.wait()
    with open("responseq2_output.txt", "r") as output_file:
        lines = output_file.readlines()
    edge_datas = []
    for line in lines:
        try:
            values = line.strip().split()
            edge_datas.append([float(value) for value in values])
        except ValueError:
            # print("ValueError in line:",line)
            pass
    edge_datas = pd.DataFrame(edge_datas,columns=['qv','q2','ex','nu','RTTOT','RLTOT','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS'])
    edge_datas.to_csv('Q2edges/'+f'Q2edge_{Q2center}.csv', index=False)
    print("Q2edge:",Q2center,"done")